# **Seggregation of the Dataset and Mean Difference Analysis**

## _ETL Workflow Notebook 5_

## Content:
1. Calculating general statistics from a given column; 
2. Getting data quantiles from a given column; 
3. Getting a particular P-percent quantile limit; 
4. Applying a list of row filters to a dataframe; 
5. Selecting subsets from a dataframe (using row filters) and labelling these subsets; 
6. Performing Analysis of Variance (ANOVA) and obtaining boxplots.

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

Install statsmodels library

In [ ]:
! pip install statsmodels

Install tensorflow library

In [ ]:
! pip install tensorflow

Install Keras library

In [ ]:
! pip install keras

Install SHAP library

In [ ]:
! pip install shap

In [ ]:
#check the version of the package
! pip show shap

In [ ]:
# Upgrade to the most recent library versions, if a given module is not present and analysis cannot be
# executed.
! pip install pip --upgrade
! pip install tensorflow --upgrade
! pip install keras --upgrade
! pip install shap --upgrade
! pip install sklearn --upgrade
! pip install pandas --upgrade
! pip install numpy --upgrade
! pip install matplotlib --upgrade
! pip install seaborn --upgrade
! pip install scipy --upgrade
! pip install statsmodels --upgrade

## **Load Python Libraries in Global Context**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.oneway import anova_oneway
import seaborn as sns

# **Function for mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [2]:
def mount_storage_system (source = 'aws', path_to_store_imported_s3_bucket = '/', s3_bucket_name = None, s3_obj_key_preffix = None):
    
    import sagemaker
    # sagemaker is AWS SageMaker Python SDK
    from sagemaker.session import Session
    from google.colab import drive
    
    # source = 'google' for mounting the google drive;
    # source = 'aws' for mounting an AWS S3 bucket.
    
    # THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # path_to_store_imported_s3_bucket: path of the Python environment to which the
    # S3 bucket contents will be imported. If it is None, or if 
    # path_to_store_imported_s3_bucket = '/', bucket will be imported to the root path. 
    # Alternatively, input the path as a string (in quotes). e.g. 
    # path_to_store_imported_s3_bucket = '/copied_s3_bucket'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_key_preffix = None. Keep it None or as an empty string (s3_obj_key_preffix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_preffix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    if (source == 'google'):
        
        print("Associate the Python environment to your Google Drive account, and authorize the access in the opened window.")
        
        drive.mount('/content/drive')
        
        print("Now your Python environment is connected to your Google Drive: the root directory of your environment is now the root of your Google Drive.")
        print("In Google Colab, navigate to the folder icon (\'Files\') of the left navigation menu to find a specific folder or file in your Google Drive.")
        print("Click on the folder or file name and select the elipsis (...) icon on the right of the name to reveal the option \'Copy path\', which will give you the path to use as input for loading objects and files on your Python environment.")
        print("Caution: save your files into different directories of the Google Drive. If files are all saved in a same folder or directory, like the root path, they may not be accessible from your Python environment.")
        print("If you still cannot see the file after moving it to a different folder, reload the environment.")
    
    elif (source == 'aws'):
        
        # Notice: if you wanted to authenticate directly from Python code, you could use
        # the following code, instead, to start the S3 client. boto3 is AWS S3 Python SDK:
        
        # import boto3
        # ACCESS_KEY = 'access_key_ID'
        # PASSWORD_KEY = 'password_key'
        # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
        # ... [here, use the same following code until line new_session = Session()]
        # [keep the line for session start. Substitute the line with the .download_data
        # method by the following line:]
        # s3_client.download_file(s3_bucket_name, s3_file_name_with_extension, path_to_store_imported_s3_bucket)
        
        # Check if the whole bucket will be downloaded (s3_obj_key_preffix = None):
        if (s3_obj_key_preffix is None):
            
            s3_obj_key_preffix = ''
        
        # If the path to store is None, also import the bucket to the root path:
        if (path_to_store_imported_s3_bucket is None):
            
            path_to_store_imported_s3_bucket = '/'
        
        # If the bucket name was provided, start the session. If not, print an error
        # message:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name to download from.")
        
        else:
        
            # start a new sagemaker session:

            print("Starting a SageMaker session to be associated with the S3 bucket.")

            new_session = Session()
            # Check sagemaker session class documentation:
            # https://sagemaker.readthedocs.io/en/stable/api/utility/session.html
            session.download_data(path = path_to_store_imported_s3_bucket, bucket = s3_bucket_name, key_prefix = s3_obj_key_preffix)

            print(f"S3 bucket contents successfully imported to path \'{path_to_store_imported_s3_bucket}\'.")
            
    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

# **Function for loading the dataframe**

In [13]:
def load_dataframe (file_directory_path, file_name_with_extension, load_txt_file_with_json_format = False, has_header = True, txt_csv_col_sep = "comma", sheet_to_load = None, json_record_path = None, json_field_separator = "_", json_metadata_preffix_list = None):
    
    import os
    import json
    import pandas as pd
    from pandas import json_normalize
    
    ## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, etc), 
    ## JSON, txt, or CSV (comma separated values) files.
    
    # file_directory_path - (string, in quotes): input the path of the directory (e.g. folder path) 
    # where the file is stored. e.g. file_directory_path = "/" or file_directory_path = "/folder"
    
    # file_name_with_extension - (string, in quotes): input the name of the file with the extension
    # e.g. file_name_with_extension = "file.xlsx", or, file_name_with_extension = "file.csv"
    
    # load_txt_file_with_json_format = False. Set load_txt_file_with_json_format = True 
    # if you want to read a file with txt extension containing a text formatted as JSON 
    # (but not saved as JSON).
    # WARNING: if load_txt_file_with_json_format = True, all the JSON file parameters of the 
    # function (below) must be set. If not, an error message will be raised.
    
    # has_header = True if the the imported table has headers (row with columns names).
    # Alternatively, has_header = False if the dataframe does not have header.
    
    ## Parameters for loading txt or CSV files
    
    # txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
    # or 'csv'. It informs how the different columns are separated.
    # Alternatively, txt_csv_col_sep = "comma" for columns separated by comma (",")
    # txt_csv_col_sep = "whitespace" for columns separated by simple spaces (" ").
    
    # sheet_to_load - This parameter has effect only when for Excel files.
    # keep sheet_to_load = None not to specify a sheet of the file, so that the first sheet
    # will be loaded.
    # sheet_to_load may be an integer or an string (inside quotes). sheet_to_load = 0
    # loads the first sheet (sheet with index 0); sheet_to_load = 1 loads the second sheet
    # of the file (index 1); sheet_to_load = "Sheet1" loads a sheet named as "Sheet1".
    # Declare a number to load the sheet with that index, starting from 0; or declare a
    # name to load the sheet with that name.
    
    ## Parameters for loading JSON files:
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_preffix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_preffix_list = ['name', 'last']
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, file_name_with_extension)
    # Extract the file extension
    file_extension = os.path.splitext(file_path)[1][1:]
    # os.path.splitext(file_path) is a tuple of strings: the first is the complete file
    # root with no extension; the second is the extension starting with a point: '.txt'
    # When we set os.path.splitext(file_path)[1], we are selecting the second element of
    # the tuple. By selecting os.path.splitext(file_path)[1][1:], we are taking this string
    # from the second character (index 1), eliminating the dot: 'txt'
    
    if ((file_extension == 'txt') | (file_extension == 'csv')): 
        # The operator & is equivalent to 'And' (intersection).
        # The operator | is equivalent to 'Or' (union).
        # pandas.read_csv method must be used.
        if (load_txt_file_with_json_format == True):
            
            print("Reading a txt file containing JSON parsed data. A reading error will be raised if you did not set the JSON parameters.")
            
            with open(file_path, 'r') as opened_file:
                # 'r' stands for read mode; 'w' stands for write mode
                # read the whole file as a string named 'file_full_text'
                file_full_text = opened_file.read()
                # if we used the readlines() method, we would be reading the
                # file by line, not the whole text at once.
                # https://stackoverflow.com/questions/8369219/how-to-read-a-text-file-into-a-string-variable-and-strip-newlines?msclkid=a772c37bbfe811ec9a314e3629df4e1e
                # https://www.tutorialkart.com/python/python-read-file-as-string/#:~:text=example.py%20%E2%80%93%20Python%20Program.%20%23open%20text%20file%20in,and%20prints%20it%20to%20the%20standard%20output.%20Output.?msclkid=a7723a1abfe811ecb68bba01a2b85bd8
                
            #Now, file_full_text is a string containing the full content of the txt file.
            json_file = json.loads(file_full_text)
            # json.load() : This method is used to parse JSON from URL or file.
            # json.loads(): This method is used to parse string with JSON content.
            # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
            # like a dataframe.
            # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
            dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
        
        else:
            # Not a JSON txt
        
            if (has_header == True):

                if (txt_csv_col_sep == "comma"):

                    dataset = pd.read_csv(file_path)

                elif (txt_csv_col_sep == "whitespace"):

                    dataset = pd.read_csv(file_path, delim_whitespace = True)

                else:
                    print(f"Enter a valid column separator for the {file_extension} file: \'comma\' or \'whitespace\'.")

            else:
                # has_header == False

                if (txt_csv_col_sep == "comma"):

                    dataset = pd.read_csv(file_path, header = None)

                elif (txt_csv_col_sep == "whitespace"):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, header = None)

                else:
                    print(f"Enter a valid column separator for the {file_extension} file: \'comma\' or \'whitespace\'.")

    elif (file_extension == 'json'):
        
        with open(file_path) as opened_file:
            
            json_file = json.load(opened_file)
            # The structure json_file = json.load(open(file_path)) relies on the GC to close the file. That's not a 
            # good idea: If someone doesn't use CPython the garbage collector might not be using refcounting (which 
            # collects unreferenced objects immediately) but e.g. collect garbage only after some time.
            # Since file handles are closed when the associated object is garbage collected or closed 
            # explicitly (.close() or .__exit__() from a context manager) the file will remain open until 
            # the GC kicks in.
            # Using 'with' ensures the file is closed as soon as the block is left - even if an exception 
            # happens inside that block, so it should always be preferred for any real application.
            # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python
            
        # json.load() : This method is used to parse JSON from URL or file.
        # json.loads(): This method is used to parse string with JSON content.
        # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.   
        dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
    
    else:
        # If it is not neither a csv nor a txt file, let's assume it is one of different
        # possible Excel files.
        print("Excel file inferred. If an error message is shown, check if a valid file extension was used: \'xlsx\', \'xls\', etc.")
            
        if (sheet_to_load is not None):        
        #Case where the user specifies which sheet of the Excel file should be loaded.
            
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load)
            
            else:
                #No header
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, header = None)
        
        else:
            #No sheet specified
            if (has_header == True):
                
                dataset = pd.read_excel(file_path)
            
            else:
                #No header
                dataset = pd.read_excel(file_path, header = None)
    
    print(f"Dataset extracted from {file_path}. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset   

# **Function for converting JSON object to dataframe**
- Objects may be:
    - String with JSON formatted text;
    - List with nested dictionaries (JSON formatted);
    - Dictionaries, possibly with nested dictionaries (JSON formatted).

In [ ]:
def json_obj_to_dataframe (json_obj_to_convert, json_record_path = None, json_field_separator = "_", json_metadata_preffix_list = None):
    
    import json
    import pandas as pd
    from pandas import json_normalize
    
    # json_obj_to_convert: object containing JSON, or string with JSON content to parse.
    # Objects may be: string with JSON formatted text;
    # list with nested dictionaries (JSON formatted);
    # dictionaries, possibly with nested dictionaries (JSON formatted).
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_preffix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_preffix_list = ['name', 'last']
    
    json_file = json.loads(json_obj_to_convert)
    # json.load() : This method is used to parse JSON from URL or file.
    # json.loads(): This method is used to parse string with JSON content.
    # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
    # like a dataframe.
    # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
    dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_preffix_list)
    
    print(f"JSON object {json_obj_to_convert} converted to a flat dataframe object. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset

# **Function for dataframe general characterization**

In [10]:
def df_gen_charac (df):
    
    import pandas as pd
    
    print("Dataframe 10 first rows:")
    print(df.head(10))
    
    #Line break before next information:
    print("\n")
    df_shape  = df.shape
    print(f"Dataframe shape (rows, columns) = {df_shape}.")
    
    #Line break before next information:
    print("\n")
    df_columns_list = df.columns
    print(f"Dataframe columns list = {df_columns_list}.")
    
    #Line break before next information:
    print("\n")
    df_dtypes = df.dtypes
    print("Dataframe variables types:")
    print(df_dtypes)
    
    #Line break before next information:
    print("\n")
    df_general_statistics = df.describe()
    print("Dataframe general statistics (numerical variables):")
    print(df_general_statistics)
    
    #Line break before next information:
    print("\n")
    df_missing_values = df.isna().sum()
    print("Total of missing values for each feature:")
    print(df_missing_values)
    
    return df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values

# **Function for column filtering (selecting) or column renaming**

In [16]:
def col_filter_rename (df, cols_list, mode = 'filter'):
    
    import pandas as pd
    
    #mode = 'filter' for filtering only the list of columns passed as cols_list;
    #mode = 'rename' for renaming the columns with the names passed as cols_list.
    
    #cols_list = list of strings containing the names (headers) of the columns to select
    # (filter); or to be set as the new columns' names, according to the selected mode.
    # For instance: cols_list = ['col1', 'col2', 'col3'] will 
    # select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
    # Declare the names inside quotes.
    
    print(f"Original columns in the dataframe:\n{df.columns}")
    
    if (mode == 'filter'):
        
        #filter the dataframe so that it will contain only the cols_list.
        df = df[cols_list]
        print("Dataframe filtered according to the list provided.")
        
    elif (mode == 'rename'):
        
        #Check if the number of columns of the dataset is equal to the number of elements
        # of the new list. It will avoid raising an exception error.
        boolean_filter = (len(cols_list) == len(df.columns))
        
        if (boolean_filter == False):
            #Impossible to rename, number of elements are different.
            print("The number of columns of the dataframe is different from the number of elements of the list. Please, provide a list with number of elements equals to the number of columns.")
        
        else:
            #Same number of elements, so that we can update the columns' names.
            df.columns = cols_list
            print("Dataframe columns renamed according to the list provided.")
            print("Warning: the substitution is element-wise: the first element of the list is now the name of the first column, and so on, ..., so that the last element is the name of the last column.")
        
        
    else:
        print("Enter a valid mode: \'filter\' or \'rename\'.")
    
    return df

# **Function for calculating general statistics from a given column**

In [3]:
def GENERAL_STATISTICS (df, column_to_analyze):
    
    import numpy as np
    import pandas as pd
    
    #df: dataframe to be analyzed.
    
    #column_to_analyze: name of the new column. e.g. column_to_analyze = 'col1'
    # will analyze column named as 'col1'
    analyzed_series = df[column_to_analyze]
    
    general_stats = analyzed_series.describe()
    print(f"General descriptive statistics from variable {column_to_analyze}, ignoring missing values:\n")
    print(general_stats)
    print("\n") #line break
    print("Interpretation: count = total of values evaluated. Missing values are ignored; mean = mean value of the series, ignoring missing values; std = standard deviation of the series, ignoring missing values; min = minimum observed value for the analyzed series; 25\% = 0.25 quantile (1st-quartile): 25\% of data fall below (<=) this value; 50\% = 2nd-quartile: 50\% <= this value; 75\% = 3rd-quartile - 75\% of data <= this value; max = maximum observed value for the analyzed series.")
    print("This function shows the general statistics only for numerical variables. The results were returned as the dataframe general_stats.")
    
    # Return only the dataframe
    return general_stats

# **Function for getting data quantiles from a given column**

In [ ]:
def GET_QUANTILES (df, column_to_analyze):
    
    import numpy as np
    import pandas as pd
    
    #df: dataframe to be analyzed.
    
    #column_to_analyze: name of the new column. e.g. column_to_analyze = 'col1'
    # will analyze column named as 'col1'
    analyzed_series = df[column_to_analyze]
    
    list_of_quantiles = []
    list_of_values = []
    list_of_interpretation = []
    
    #First element: minimum
    list_of_quantiles.append(0)
    list_of_values.append(analyzed_series.min())
    list_of_interpretation.append(f"Minimum value of {column_to_analyze}")
    
    i = 0.05
    #Start from the 5% quantile
    while (i <= 0.95):
        
        list_of_quantiles.append(i)
        list_of_values.append(analyzed_series.quantile(i))
        list_of_interpretation.append(f"{i}-quantile: {i*100}\% of data fall below this value")
        
        i = i + 0.05
    
    # Last element: maximum value
    list_of_quantiles.append(1)
    list_of_values.append(analyzed_series.max())
    list_of_interpretation.append(f"Maximum value of {column_to_analyze}")
    
    # Summarize the lists as a dictionary and convert the dictionary to a dataframe:
    quantiles_dict = {"quantile": list_of_quantiles, f"value_of_{column_to_analyze}": list_of_values,
                     "interpretation": list_of_interpretation}
    
    quantiles_summ_df = pd.DataFrame(data = quantiles_dict)
    
    print("Quantiles returned as dataframe quantiles_summ_df. Check it below:\n")
    print(quantiles_summ_df)
    
    return quantiles_summ_df

# **Function for getting a particular P-percent quantile limit**
- input P as a percent, from 0 to 100.

In [2]:
def GET_P_PERCENT_QUANTILE_LIM (df, column_to_analyze, p_percent = 100):
    
    import numpy as np
    import pandas as pd
    
    #df: dataframe to be analyzed.
    
    #column_to_analyze: name of the new column. e.g. column_to_analyze = 'col1'
    # will analyze column named as 'col1'
    
    # p_percent: float value from 0 to 100 representing the percent of the quantile
    # if p_percent = 31.2, then 31.2% of the data will fall below the returned value
    # if p_percent = 75, then 75% of the data will fall below the returned value
    # if p_percent = 0, the minimum value is returned.
    # if p_percent = 100, the maximum value is returned.
    
    analyzed_series = df[column_to_analyze]
    
    #convert the quantile to fraction
    quantile_fraction = p_percent/100.0 #.0 to guarantee a float result
    
    if (quantile_fraction < 0):
        print("Invalid percent value - it cannot be lower than zero.")
        return "error"
    
    elif (quantile_fraction == 0):
        #get the minimum value
        quantile_lim = analyzed_series.min()
        print(f"Minimum value of {column_to_analyze} = {quantile_lim}")
    
    elif (quantile_fraction == 1):
        #get the maximum value
        quantile_lim = analyzed_series.max()
        print(f"Maximum value of {column_to_analyze} = {quantile_lim}")
        
    else:
        #get the quantile
        quantile_lim = analyzed_series.quantile(quantile_fraction)
        print(f"{quantile_fraction}-quantile: {p_percent}\% of data fall below this value")
    
    return quantile_lim

# **Function for applying a list of row filters to a dataframe**

In [ ]:
def APPLY_ROW_FILTERS_LIST (df, list_of_row_filters):
    
    import numpy as np
    import pandas as pd
    
    print("Warning: this function filter the rows and results into a smaller dataset, since it removes the non-selected entries.")
    print("If you want to pass a filter to simply label the selected rows, use the function LABEL_DATAFRAME_SUBSETS, which do not eliminate entries from the dataframe.")
    
    # This function applies filters to the dataframe and remove the non-selected entries.
    
    # df: dataframe to be analyzed.
    
    ## define the filters and only them define the filters list
    # EXAMPLES OF BOOLEAN FILTERS TO COMPOSE THE LIST
    # boolean_filter1 = ((None) & (None)) 
    # (condition1 AND (&) condition2)
    # boolean_filter2 = ((None) | (None)) 
    # condition1 OR (|) condition2
    
    # boolean filters result into boolean values True or False.

    ## Examples of filters:
    ## filter1 = (condition 1) & (condition 2)
    ## filter1 = (df['column1'] > = 0) & (df['column2']) < 0)
    ## filter2 = (condition)
    ## filter2 = (df['column3'] <= 2.5)
    ## filter3 = (df['column4'] > 10.7)
    ## filter3 = (condition 1) | (condition 2)
    ## filter3 = (df['column5'] != 'string1') | (df['column5'] == 'string2')

    ## comparative operators: > (higher); >= (higher or equal); < (lower); 
    ## <= (lower or equal); == (equal); != (different)

    ## concatenation operators: & (and): the filter is True only if the 
    ## two conditions concatenated through & are True
    ## | (or): the filter is True if at least one of the two conditions concatenated
    ## through | are True.
    
    ## Pandas .isin method: you can also use this method to filter rows belonging to
    ## a given subset (the row that is in the subset is selected). The syntax is:
    ## is_black_or_brown = dogs["color"].isin(["Black", "Brown"])
    ## or: filter = (dataframe_column_series).isin([value1, value2, ...])

    ## separate conditions with parentheses. Use parentheses to define a order
    ## of definition of the conditions:
    ## filter = ((condition1) & (condition2)) | (condition3)
    ## Here, firstly ((condition1) & (condition2)) = subfilter is evaluated. 
    ## Then, the resultant (subfilter) | (condition3) is evaluated.
    
    # list_of_row_filters: list of boolean filters to be applied to the dataframe
    # e.g. list_of_row_filters = [filter1]
    # applies a single filter saved as filter 1. Notice: even if there is a single
    # boolean filter, it must be declared inside brackets, as a single-element list.
    # That is because the function will loop through the list of filters.
    # list_of_row_filters = [filter1, filter2, filter3, filter4]
    # will apply, in sequence, 4 filters: filter1, filter2, filter3, and filter4.
    # Notice that the filters must be declared in the order you want to apply them.
    
    
    #Loop through the filters list, applying the filters sequentially:
    # Each element of the list is identified as 'boolean_filter'
    for boolean_filter in list_of_row_filters:
        
        #Apply the filter:
        df = df[boolean_filter]
    
    print("Successfully filtered the dataframe. Check the 10 first rows of the filtered and returned dataframe:\n")
    print(df.head(10))
    
    return df

# **Function for selecting subsets from a dataframe (using row filters) and labelling these subsets**

In [ ]:
def LABEL_DATAFRAME_SUBSETS (df, list_of_row_filters, new_labels_list = None, new_labelled_column_name = None):
    
    import numpy as np
    import pandas as pd
    
    print("Attention: this function selects subsets from the dataframe and label them, allowing the seggregation of the data.")
    print("If you want to filter the dataframe to eliminate non-selected rows, use the function APPLY_ROW_FILTERS_LIST")
    
    ## This function selects subsets of the dataframe by applying a list
    ## of row filters, and then it labels each one of the filtered subsets.
    
    # df: dataframe to be analyzed.
    
    ## define the filters and only them define the filters list
    # EXAMPLES OF BOOLEAN FILTERS TO COMPOSE THE LIST
    # boolean_filter1 = ((None) & (None)) 
    # (condition1 AND (&) condition2)
    # boolean_filter2 = ((None) | (None)) 
    # condition1 OR (|) condition2
    
    # boolean filters result into boolean values True or False.

    ## Examples of filters:
    ## filter1 = (condition 1) & (condition 2)
    ## filter1 = (df['column1'] > = 0) & (df['column2']) < 0)
    ## filter2 = (condition)
    ## filter2 = (df['column3'] <= 2.5)
    ## filter3 = (df['column4'] > 10.7)
    ## filter3 = (condition 1) | (condition 2)
    ## filter3 = (df['column5'] != 'string1') | (df['column5'] == 'string2')

    ## comparative operators: > (higher); >= (higher or equal); < (lower); 
    ## <= (lower or equal); == (equal); != (different)

    ## concatenation operators: & (and): the filter is True only if the 
    ## two conditions concatenated through & are True
    ## | (or): the filter is True if at least one of the two conditions concatenated
    ## through | are True.
    
    ## Pandas .isin method: you can also use this method to filter rows belonging to
    ## a given subset (the row that is in the subset is selected). The syntax is:
    ## is_black_or_brown = dogs["color"].isin(["Black", "Brown"])
    ## or: filter = (dataframe_column_series).isin([value1, value2, ...])

    ## separate conditions with parentheses. Use parentheses to define a order
    ## of definition of the conditions:
    ## filter = ((condition1) & (condition2)) | (condition3)
    ## Here, firstly ((condition1) & (condition2)) = subfilter is evaluated. 
    ## Then, the resultant (subfilter) | (condition3) is evaluated.
    
    # list_of_row_filters: list of boolean filters to be applied to the dataframe
    # e.g. list_of_row_filters = [filter1]
    # applies a single filter saved as filter 1. Notice: even if there is a single
    # boolean filter, it must be declared inside brackets, as a single-element list.
    # That is because the function will loop through the list of filters.
    # list_of_row_filters = [filter1, filter2, filter3, filter4]
    # will apply, in sequence, 4 filters: filter1, filter2, filter3, and filter4.
    # Notice that the filters must be declared in the order you want to apply them.
        
    # Here, each filter will be associated with one label. Therefore, if you want to apply 
    # a filter with several conditions, combine them using the & and | operators.
    
    # The labels are input as a list of strings or of discrete integer values. There must be
    # exactly one filter to each label: the filter will be used to obtain a subset from
    # the dataframe, and the label will be applied to all the rows of this subset.
    # e.g. list_of_row_filters = [filter1, filter2], new_labels_list = ['label1', 'label2']
    # In this case, the rows of the dataset obtained when applying filter1 will receive the
    # string 'label1' as label. In turns, 
    # rows selected from filter2 will be labelled as 'label2'.
    # list_of_row_filters = [filter1, filter2, filter3], new_labels_list = [1, 2, 3]
    # Here, the labels are integers: rows filtered through filter1 are marked as 1;
    # rows selected by filter2 are labelled as 2; and rows selected from filter3 are marked
    # as 3.
    
    # Warning: the sequence of filters must be correspondent to the sequence of labels. 
    # Rows selected from the first filter are labelled with the first item from the labels
    # list; rows selected by the 2nd filter are labelled with the 2nd element, and so on.
    
    # If no list of labels is provided, a list of integers starting from zero and with the
    # same total of elements from list_of_row_filters will be created. For instance,
    # if list_of_row_filters = [filter1, filter2, filter3, filter4], the list of labels
    # [0, 1, 2, 3] would be created. Again, the rows selected from filter1 would be labelled
    # as 0; rows selected from filter 2 would be labelled as 1; and so on.
    
    # new_labelled_column_name = None. Alternatively, declare a string (inside quotes), with
    # the name of the column containing the applied labels. For example, if
    # new_labelled_column_name = 'labels', then the new labels are saved in the column 'labels'
    # If no new_labelled_column_name is provided, the default name = "df_label" is provided.
    
    if (new_labelled_column_name is None):
        
        new_labelled_column_name = "df_label"
    
    if (new_labels_list is None):
        
        #Create a list of integers with the same length as list_of_row_filters
        # start the list:
        new_labels_list = []
        # Append the value zero:
        new_labels_list.append(0)
        
        #check the total of elements of list list_of_row_filters:
        total_of_filters = len(list_of_row_filters)
        
        #loop from i = 1 to i = (total_of_filters - 1), index of the last element from
        #list total_of_filters. Append these values to the list of new labels:
        for i in range (1, total_of_filters):
            #i goes from 1 to (total_of_filters - 1).
            # the element 0 was already added
            new_labels_list.append(i)
            # now we have a list of integers from 0 to (total_of_filters - 1), where each
            # number is a label to be applied to the rows selected from one of the filters.
        
    # Let's select rows applying the filters, and label them according    
    
    #Loop through the filters list, applying the filters sequentially:
    # Each element of the list is identified as 'boolean_filter'
    for i in range (0, total_of_filters):
        # Here, i ranges from 0 to total_of_filters - 1, index of the last element
        # We specifically declare 0 as the first element to avoid problems derived
        # from the fact that a counting variable i was already used and could have a
        # value in the memory. The value 0 is the default when only one value is declared
        # as argument from range
        
        #select the boolean filter from list_of_row_filters. It is the i-th indexed element 
        boolean_filter = list_of_row_filters[i]
        # Select the correspondent label from new_labels_list:
        applied_label = new_labels_list[i]
        
        #Apply the filter to select a group of rows, and apply the correspondent label
        # to the selected rows
        
        # syntax: dataset.loc[dataset['column_filtered'] <= 0.87, 'labelled_column'] = 1
        # which is equivalent to dataset.loc[(filter), 'labelled_column'] = label
        df = df.loc[(boolean_filter), new_labelled_column_name] = applied_label
    
    print("Successfully labelled the dataframe. Check the 10 first rows of the labelled and returned dataframe:\n")
    print(df.head(10))
    
    return df

# **Function for performing Analysis of Variance (ANOVA) and obtaining boxplots**

In [61]:
def anova_boxplot (df, column_to_analyze, column_with_group_labels = None, confidence_level = 0.95, boxplot_notch = False, boxplot_vertically_oriented = True, boxplot_patch_artist = False,  reference_value = None, x_axis_rotation = 0, y_axis_rotation = 0, grid = True, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
        
        print ("If an error message is shown, update statsmodels. Declare and run a cell as:")
        print ("!pip install statsmodels --upgrade")
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        from statsmodels.stats.oneway import anova_oneway
        
        # df: dataframe to be analyzed.
    
        #column_to_analyze: name of the new column. e.g. column_to_analyze = 'col1'
        # will analyze column named as 'col1'
        
        # column_with_group_labels: name of the column registering groups, labels, or
        # factors correspondent to each row. e.g. column_with_group_labels = "group"
        # if the correspondent group is saved as column named 'group'.
        # If no name is provided, then the default name is used:
        # COLUMN_WITH_GROUP_LABELS = COLUMN_TO_ANALYZE + "_label"
        
        if (column_with_group_labels is None):
            
            column_with_group_labels = column_to_analyze + "_label"
    
        # CONFIDENCE_LEVEL = 0.95 = 95% confidence
        # Set CONFIDENCE_LEVEL = 0.90 to get 0.90 = 90% confidence in the analysis.
        # Notice that, when less trust is needed, we can reduce CONFIDENCE_LEVEL 
        # to get less restrictive results.
        alpha = 1 - confidence_level
        
        # reference_value: keep it as None or add a float value.
        # This reference value will be shown as a red constant line on the plot to be compared
        # with the boxes. e.g. reference_value = 1.0 will plot a red line passing through
        # column_to_analyze = 1.0
        
        # boxplot_notch = False
        # Manipulate parameter notch (boolean, default: False) from the boxplot object
        # Whether to draw a notched boxplot (True), or a rectangular boxplot (False). 
        # The notches represent the confidence interval (CI) around the median. 
        # The documentation for bootstrap describes how the locations of the notches are 
        # computed by default, but their locations may also be overridden by setting the 
        # conf_intervals parameter.
        
        # boxplot_vertically_oriented = True
        # Manipulate the parameter vert (boolean, default: True)
        # If True, draws vertical boxes. If False, draw horizontal boxes.
        
        # boxplot_patch_artist = False
        # Manipulate parameter patch_artist (boolean, default: False)
        # If False produces boxes with the Line2D artist. Otherwise, boxes are drawn 
        # with Patch artists.
        # Check documentation:
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html
        
        # Create a local copy of the dataframe to be manipulated:
        DATASET = df
        
        # Sort the dataframe by the groups/factors column (column_with_group_labels):
        DATASET = DATASET.sort_values(column_with_group_labels, ascending = True)
        
        # Reset the index of the new dataframe:
        DATASET = DATASET.reset_index(drop = True)
        
        # Get a series for the analyzed values and for the 
        # column with the correspondent labels:
        analyzed_series = DATASET[column_to_analyze]
        groups_series = DATASET[column_with_group_labels]
        
        # Get a series of unique values of groups
        # Convert the series of columns into a NumPy array
        array_of_groups = np.array(groups_series)
        
        #Keep only the unique values into this array:
        array_of_groups = np.unique(array_of_groups)
        
        #Get the total of groups
        total_of_groups = len(array_of_groups)
        
        
        # Create a dictionary of NumPy arrays. This dictionary will store NumPy arrays
        # correspondent to the elements of each group:
        values_for_each_group_dict = {}
        
        # Loop through each element of the array_of_groups (array with unique values).
        # The elements of array_of_groups are identified as 'group'
        for group in array_of_groups:
            
            #start a list to store the values:
            analyzed_vals_list = []
            
            # Now loop through each row of the dataframe. The dataframe row index go from
            # zero to len(DATASET) - 1:
            for i in range((len(DATASET) - 1)):
                #loop go from i = 0 to i = len(DATASET) - 1, index of the last row
                
                # if the row in index is correspondent to the grouping/label 'group',
                # then store the analyzed value into list analyzed_vals_list:
                if (groups_series[i] == group):
                    
                    analyzed_vals_list.append(analyzed_series[i])
            
            # lets convert the lists into NumPy arrays:
            analyzed_vals_array = np.array(analyzed_vals_list)
            
            # Now let's update the dictionary values_for_each_group_dict
            # 1. Set the 'key' of the dictionary as the name of the group
            dict_key = group
            
            # Use the update method to add the key 'dict_key', and the 
            # array analyzed_vals_array as the value correspondent to the key.
            
            # syntax: dict.update({"key": value})
            # check: https://www.w3schools.com/python/ref_dictionary_update.asp
            
            values_for_each_group_dict.update({dict_key: analyzed_vals_array})
        
        # Now, the dictionary values_for_each_group_dict stores each group name as its keys
        # and the arrays of analyzed values as the correspondent object.
        
        # Notice that the keys of the dictionary were created with the same order of
        # array_of_groups, so there is no risk of losing the correspondence
        
        # Create a humongous list of arrays, containing the arrays stored into the
        # dictionary. It is necessary for the statsmodels API:
        humongous_list = []
        
        # Loop through each key of the dictionary:
        for group in values_for_each_group_dict.keys():
            # loop through each element of the array of keys from the dictionary.
            # each element is named 'group'.
            # append the correspondent array as an element from humongous_list
            humongous_list.append(values_for_each_group_dict[group])
        
        #Now, we can pass the humongous_list as argument for the one-way Anova:
        anova_one_way_summary = anova_oneway(humongous_list, groups = array_of_groups, use_var = 'bf', welch_correction=True, trim_frac=0)
        # When use_var = 'bf', variances are not assumed to be equal across samples.
        # Check documentation: 
        # https://www.statsmodels.org/stable/generated/statsmodels.stats.oneway.anova_oneway.html
        
        # The information is stored in a tuple (f_statistic, p-value)
        # f_statistic: Test statistic for k-sample mean comparison which is approximately 
        # F-distributed.
        # p-value: If use_var="bf", then the p-value is based on corrected degrees of freedom following Mehrotra 1997.
        f_statistic = anova_one_way_summary[0]
        p_value = anova_one_way_summary[1]
        
        print(f"Probability that the means of the groups are the same = {100*p_value}\% (p-value = {p_value})")
        print(f"Calculated F-statistic for the variances = {f_statistic}")
        
        if (p_value <= alpha):
            print(f"For a confidence level of {confidence_level*100}\%, we can reject the null hypothesis.")
            print(f"The means are different for a {confidence_level*100}\% confidence level.")
        
        else:
            print(f"For a confidence level of {confidence_level*100}\%, we can accept the null hypothesis.")
            print(f"The means are equal for a {confidence_level*100}\% confidence level.")
        
        print("Check ANOVA summary:\n")
        print(anova_one_way_summary)
        # When printing the summary, the set of supporting information of the Tuple are shown.
        
        anova_summary_dict = {'F_statistic': f_statistic, 'p_value': p_value}
        
        #Now, let's obtain the boxplot
        fig, ax = plt.subplots()
                    
        # rectangular box plot
        # The arrays of each group are the elements of the list humongous_list
        boxplot_returned_dict = ax.boxplot(humongous_list, labels = array_of_groups, notch = boxplot_notch, vert = boxplot_vertically_oriented, patch_artist = boxplot_patch_artist)
        
        # boxplot_returned_dict: A dictionary mapping each component of the boxplot to 
        # a list of the Line2D instances created. That dictionary has the following keys 
        # (assuming vertical boxplots):
        # boxes: the main body of the boxplot showing the quartiles and the median's 
        # confidence intervals if enabled.
        # medians: horizontal lines at the median of each box.
        # whiskers: the vertical lines extending to the most extreme, non-outlier data 
        # points.
        # caps: the horizontal lines at the ends of the whiskers.
        # fliers: points representing data that extend beyond the whiskers (fliers).
        # means: points or lines representing the means.
          
        ax.set_title(f"Boxplot of {column_to_analyze} by {col_with_group_indication}")
        
        if (boxplot_vertically_oriented == True):
            # generate vertically-oriented boxplot
        
            ax.set(ylabel = column_to_analyze)
            # If the boxplot was horizontally oriented, this label should be the X instead.
            # The X labels were already defined when creating the boxplot
                    
            if not (reference_value is None):
                # Add an horizontal reference_line to compare against the boxes:
                # If the boxplot was horizontally-oriented, this line should be vertical instead.
                ax.axhline(reference_value, color = 'red', linestyle = 'dashed', label = 'Reference value')
                # axhline generates an horizontal (h) line on ax
                
        else:
            # In case it is False, generate horizontally-oriented plot:
            ax.set(xlabel = column_to_analyze)
            # The Y labels were already defined when creating the boxplot
                    
            if not (reference_value is None):
                # Add an horizontal reference_line to compare against the boxes:
                # If the boxplot was horizontally-oriented, this line should be vertical instead.
                ax.axvline(reference_value, color = 'red', linestyle = 'dashed', label = 'Reference value')
                # axvline generates a vertical (v) line on ax
        
            # Now, we can add general components of the graphic:
            #Create an almost transparent horizontal line to guide the eyes without distracting
            ax.yaxis.grid(grid, linestyle='-', which = 'major', color = 'lightgrey', alpha = 0.5)
            ax.xaxis.grid(grid, linestyle='-', which = 'major', color = 'lightgrey', alpha = 0.5)       
            
            #ROTATE X AXIS IN XX DEGREES
            plt.xticks(rotation = x_axis_rotation)
            # XX = 70 DEGREES x_axis (Default)
            #ROTATE Y AXIS IN XX DEGREES:
            plt.yticks(rotation = y_axis_rotation)
            # XX = 0 DEGREES y_axis (Default)
            ax.legend()

            if (export_png == True):
                # Image will be exported
                import os

                #check if the user defined a directory path. If not, set as the default root path:
                if (directory_to_save is None):
                    #set as the default
                    directory_to_save = "/"

                #check if the user defined a file name. If not, set as the default name for this
                # function.
                if (file_name is None):
                    #set as the default
                    file_name = "boxplot"

                #check if the user defined an image resolution. If not, set as the default 110 dpi
                # resolution.
                if (png_resolution_dpi is None):
                    #set as 110 dpi
                    png_resolution_dpi = 110

                #Get the new_file_path
                new_file_path = os.path.join(directory_to_save, file_name)

                #Export the file to this new path:
                # The extension will be automatically added by the savefig method:
                plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
                #quality could be set from 1 to 100, where 100 is the best quality
                #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
                #transparent = True or False
                # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
                print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

            #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
            plt.figure(figsize=(12, 8))
            #fig.tight_layout()

            ## Show an image read from an image file:
            ## import matplotlib.image as pltimg
            ## img=pltimg.imread('mydecisiontree.png')
            ## imgplot = plt.imshow(img)
            ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
            ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
            ##  '03_05_END.ipynb'
            plt.show()
            
            print("\n") #line break
            print("Successfully returned 2 dictionaries: anova_summary_dict (dictionary storing ANOVA F-test and p-value); and boxplot_returned_dict (dictionary mapping each component of the boxplot).\n")
            
            print("Boxplot interpretation:")
            print("Boxplot presents the following key visual components:")
            print("The main box represents the Interquartile Range (IQR). It represents the data that is from quartile Q1 to quartile Q3.")
            print("Q1 = 1st quartile of the dataset. 25% of values lie below this level (i.e., it is the 0.25-quantile or percentile).")
            print("Q2 = 2nd quartile of the dataset. 50% of values lie above and below this level (i.e., it is the 0.50-quantile or percentile).")
            print("Q3 = 3rd quartile of the dataset. 75% of values lie below and 25% lie above this level (i.e., it is the 0.75-quantile or percentile).")
            print("Boxplot main box (the IQR) is divided by an horizontal line if it is vertically-oriented; or by a vertical line if it is horizontally-oriented.")
            print("This line represents the median: it is the midpoint of the dataset.")
            print("There are lines extending beyond the main boxes limits. This lines end in horizontal limits, if the boxplot is vertically oriented; or in vertical limits, for an horizontal plot.")
            print("The minimum limit of the boxplot is defined as: Q1 - (1.5) x (IQR width) = Q1 - 1.5*(Q3-Q1)")
            print("The maximum limit of the boxplot is defined as: Q3 + (1.5) x (IQR width) = Q3 + 1.5*(Q3-Q1)")
            print("Finally, there are isolated points (circles) on the plot.")
            print("These points lie below the minimum bar, or above the maximum bar line. They are defined as outliers.")
            # https://nickmccullum.com/python-visualization/boxplot/
            
            return anova_summary_dict, boxplot_returned_dict

# **Function for concatenating (SQL UNION) multiple dataframes**
- Vertical concatenation of the dataframes.
- Equivalent to SQL Union: vertical stack/append of the tables.

In [50]:
def UNION_DATAFRAMES (list_of_dataframes, what_to_append = 'rows', ignore_index_on_union = True, sort_values_on_union = True, union_join_type = None):
    
    import pandas as pd
    #JOIN can be 'inner' to perform an inner join, eliminating the missing values
    #The default (None) is 'outer': the dataframes will be stacked on the columns with
    #same names but, in case there is no correspondence, the row will present a missing
    #value for the columns which are not present in one of the dataframes.
    #When using the 'inner' method, only the common columns will remain
    
    #list_of_dataframes must be a list containing the dataframe objects
    # example: list_of_dataframes = [df1, df2, df3, df4]
    #Notice that the dataframes are objects, not strings. Therefore, they should not
    # be declared inside quotes.
    # There is no limit of dataframes. In this example, we will concatenate 4 dataframes.
    # If list_of_dataframes = [df1, df2, df3] we would concatenate 3, and if
    # list_of_dataframes = [df1, df2, df3, df4, df5] we would concatenate 5 dataframes.
    
    # what_to_append = 'rows' for appending the rows from one dataframe
    # into the other; what_to_append = 'columns' for appending the columns
    # from one dataframe into the other (horizontal or lateral append).
    
    # When what_to_append = 'rows', Pandas .concat method is defined as
    # axis = 0, i.e., the operation occurs in the row level, so the rows
    # of the second dataframe are added to the bottom of the first one.
    # It is the SQL union, and creates a dataframe with more rows, and
    # total of columns equals to the total of columns of the first dataframe
    # plus the columns of the second one that were not in the first dataframe.
    # When what_to_append = 'columns', Pandas .concat method is defined as
    # axis = 1, i.e., the operation occurs in the column level: the two
    # dataframes are laterally merged using the index as the key, 
    # preserving all columns from both dataframes. Therefore, the number of
    # rows will be the total of rows of the dataframe with more entries,
    # and the total of columns will be the sum of the total of columns of
    # the first dataframe with the total of columns of the second dataframe.
    
    #The other parameters are the same from Pandas .concat method.
    # ignore_index_on_union = ignore_index;
    # sort_values_on_union = sort
    # union_join_type = join
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    
    #Check Datacamp course Joining Data with pandas, Chap.3, 
    # Advanced Merging and Concatenating
    
    # Check axis:
    if (what_to_append == 'rows'):
        
        AXIS = 0
    
    elif (what_to_append == 'columns'):
        
        AXIS = 1
    
    else:
        print("No valid string was input to what_to_append, so appending rows (vertical append, equivalent to SQL UNION).")
        AXIS = 0
    
    if (union_join_type == 'inner'):
        
        print("Warning: concatenating dataframes using the \'inner\' join method, that removes missing values.")
        concat_df = pd.concat(list_of_dataframes, axis = AXIS, ignore_index = ignore_index_on_union, sort = sort_values_on_union, join = union_join_type)
    
    else:
        
        #In case None or an invalid value is provided, use the default 'outer', by simply
        # not declaring the 'join':
        concat_df = pd.concat(list_of_dataframes, axis = AXIS, ignore_index = ignore_index_on_union, sort = sort_values_on_union)
    
    # Pandas .head(Y) method results in a dataframe containing the first Y rows of the 
    # original dataframe. The default .head() is Y = 5. Print first 10 rows of the 
    # new dataframe:
    print("Dataframes successfully concatenated. Check the 10 first rows of new dataframe:\n")
    print(concat_df.head(10))
    
    #Now return the concatenated dataframe:
    
    return concat_df

# **Function for time series visualization**
        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [13]:
def time_series_vis (x1 = None, y1 = None, x2 = None, y2 = None, x3 = None, y3 = None, x4 = None, y4 = None, x5 = None, y5 = None, x6 = None, y6 = None, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, add_splines_lines = True, add_scatter_dots = False, lab1 = None, lab2 = None, lab3 = None, lab4 = None, lab5 = None, lab6 = None, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    if (add_splines_lines == True):
        line_value = '-'
    else:
        line_value = ''
    
    if (add_scatter_dots == True):
        marker_value = 'o'
    else:
        marker_value = ''
    
    fig = plt.figure()
    ax = fig.add_subplot()
    
    if not (lab1 is None):
        
        label_1 = lab1
    
    else:
        label_1 = "Y1"

    if not (x1 is None):
        ax.plot(x1, y1, linestyle = line_value, marker = marker_value, color='blue', label=label_1)
    
    if not (x2 is None):
        #runs only when both are present
        if not (lab2 is None):
            label_2 = lab2
        else:
            label_2 = "Y2"
        
        ax.plot(x2, y2, linestyle = line_value, marker = marker_value, color='red', label=label_2)
    
    if not (x3 is None):
                
        if not (lab3 is None):
            label_3 = lab3
        else:
            label_3 = "Y3"
        
        ax.plot(x3, y3, linestyle = line_value, marker = marker_value, color='green', label=label_3)
    
    if not (x4 is None):
                
        if not (lab4 is None):
            label_4 = lab4
        else:
            label_4 = "Y4"
        
        ax.plot(x4, y4, linestyle = line_value, marker = marker_value, color='black', label=label_4)
    
    if not (x5 is None):
               
        if not (lab5 is None):
            label_5 = lab5
        else:
            label_5 = "Y5"
        
        ax.plot(x5, y5, linestyle = line_value, marker = marker_value, color='magenta', label=label_5)
   
    if not (x6 is None):
               
        if not (lab6 is None):
            label_6 = lab6
        else:
            label_6 = "Y6"
        
        ax.plot(x6, y6, linestyle = line_value, marker = marker_value, color='yellow', label=label_6)
   
    if not (plot_title is None):
        #graphic's title
        ax.set_title(plot_title) 
    
    if not (horizontal_axis_title is None):
        #X-axis title
        ax.set_xlabel(horizontal_axis_title)
    
    if not (vertical_axis_title is None):
        #Y-axis title
        ax.set_ylabel(vertical_axis_title)
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    ax.grid(grid)
    ax.legend()
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "time_series_vis"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()

# **Functions for histogram visualization**
- Function `histogram`: ideal bin interval is calculated through Montgomery's method. Histogram is obtained from this calculated bin size.
    - Douglas C. Montgomery (2009). Introduction to Statistical Process Control, Sixth Edition, John Wiley & Sons.
- Function `histogram_alternative`: histogram is obtained by manually defining the total of bins (i.e., into how much intervals the sample space should be divided).

In [27]:
def histogram (y, bar_width, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, normal_curve_overlay = True, data_units_label = None, y_title = None, histogram_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    
    # ideal bin interval calculated through Montgomery's method. 
    # Histogram is obtained from this calculated bin size.
    # Douglas C. Montgomery (2009). Introduction to Statistical Process Control, 
    # Sixth Edition, John Wiley & Sons.
    
    
    #Calculo do bin size - largura do histograma:
    #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
    #2: Calcular rangehist = highest - lowest
    #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
    #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
    #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
    #5: Calcular binsize = rangehist/(ncells)
    #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
    #isso porque a largura do histograma tem que ser um numero positivo

    y = y.reset_index(drop=True)
    #faz com que os indices desta serie sejam consecutivos e a partir de zero

    #Estatisticas gerais: media (mu) e desvio-padrao (sigma)
    mu = y.mean() 
    sigma = y.std() 

    #Calculo do bin-size
    highest = y.max()
    lowest = y.min()
    rangehist = highest - lowest
    rangehist = abs(rangehist)
    #garante que sera um numero positivo
    samplesize = y.count() #contagem do total de entradas
    ncells = (samplesize)**0.5 #potenciacao: ** - raiz quadrada de samplesize
    #resultado da raiz quadrada e sempre positivo
    ncells = round(ncells) #numero "redondo" mais proximo
    ncells = int(ncells) #parte inteira do numero arredondado
    #ncells = numero de linhas da tabela de frequencias
    binsize = rangehist/ncells
    binsize = round(binsize)
    binsize = int(binsize) #precisa ser inteiro
    
    #Construcao da tabela de frequencias

    j = 0 #indice da tabela de frequencias
    #Este indice e diferente do ordenamento dos valores em ordem crescente
    xhist = []
    #Lista vazia que contera os x do histograma
    yhist = []
    #Listas vazia que conteras o y do histograma
    hist_labels = []
    #Esta lista gravara os limites da barra na forma de strings

    pontomediodabarra = lowest + binsize/2 
    limitedabarra = lowest + binsize
    #ponto medio da barra 
    #limite da primeira barra do histograma
    seriedohist1 = y
    seriedohist1 = seriedohist1.sort_values(ascending=True)
    #serie com os valores em ordem crescente
    seriedohist1 = seriedohist1.reset_index(drop=True)
    #garante que a nova serie tenha indices consecutivos, iniciando em zero
    i = 0 #linha inicial da serie do histograma em ordem crescente
    valcomparado = seriedohist1[i]
    #primeiro valor da serie, o mais baixo

    while (j <= (ncells-1)):
        
        #para quando termina o numero de linhas da tabela
        xhist.append(pontomediodabarra)
        #tempo da tabela de frequencias
        cont = 0
        #variavel de contagem do histograma
        #contagem deve ser reiniciada
       
        if (i < samplesize):
            #2 condicionais para impedir que um termo de indice inexistente
            #seja acessado
            while (valcomparado <= limitedabarra) and (valcomparado < highest):
                #o segundo criterio garante a parada em casos em que os dados sao
                #muito proximos
                    cont = cont + 1 #adiciona contagem a tabela de frequencias
                    i = i + 1
                    
                    if (i < samplesize): 
                        valcomparado = seriedohist1[i]
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias
        #%.2f: 2 casas decimais de aproximação
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
        j = j + 1
    
    #Temos que verificar se o valor maximo foi incluido
    #isso porque o processo de aproximacao por numero inteiro pode ter
    #arredondado para baixo e excluido o limite superior
    #Porem, note que na ultima iteracao o limite superior da barra foi 
    #somado de binsize, mas como j ja e maior que ncells-1, o loop parou
    
    #assim, o limitedabarra nesse momento e o limite da barra que seria
    #construida em seguida, nao da ultima barra da tabela de frequencias
    #isso pode fazer com que esta barra ja seja maior que o highest
    
    #note porem que nao aumentamos o valor do limite inferior da barra
    #por isso, basta vermos se ele mais o binsize sao menores que o valor mais alto
        
    while ((limite_infdabarra+binsize) < highest):
        
        #vamos criar novas linhas ate que o ponto mais alto do histograma
        #tenha sido contado
        ncells = ncells + 1 #adiciona uma linha a tabela de frequencias
        xhist.append(pontomediodabarra)
        
        cont = 0 #variavel de contagem do histograma
        
        while (valcomparado <= limitedabarra):
                cont = cont + 1 #adiciona contagem a tabela de frequencias
                i = i + 1
                if (i < samplesize):
                    valcomparado = seriedohist1[i]
                    #apenas se i ainda nao e maior que o total de dados
                
                else: 
                    
                    break
        
        #parar o loop se i atingiu um tamanho maior que a quantidade 
        #de dados.Temos que ter este cuidado porque estamos acrescentando
        #mais linhas a tabela de frequencias para corrigir a aproximacao
        #de ncells por um numero inteiro
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias - 2 casas decimais
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
    estatisticas_col1 = []
    #contera as descricoes das colunas da tabela de estatisticas gerais
    estatisticas_col2 = []
    #contera os valores da tabela de estatisticas gerais
    
    estatisticas_col1.append("Count of data evaluated")
    estatisticas_col2.append(samplesize)
    estatisticas_col1.append("Average (mu)")
    estatisticas_col2.append(mu)
    estatisticas_col1.append("Standard deviation (sigma)")
    estatisticas_col2.append(sigma)
    estatisticas_col1.append("Highest value")
    estatisticas_col2.append(highest)
    estatisticas_col1.append("Lowest value")
    estatisticas_col2.append(lowest)
    estatisticas_col1.append("Data range (maximum value - lowest value)")
    estatisticas_col2.append(rangehist)
    estatisticas_col1.append("Bin size (bar width)")
    estatisticas_col2.append(binsize)
    estatisticas_col1.append("Total rows in frequency table")
    estatisticas_col2.append(ncells)
    #como o comando append grava linha a linha em sequencia, garantimos
    #a correspondencia das colunas
    #Assim como em qualquer string, incluindo de rotulos de graficos
    #os \n sao lidos como quebra de linha
    
    d1 = {"General Statistics": estatisticas_col1, "Calculated Value": estatisticas_col2}
    #dicionario das duas series, para criar o dataframe com as descricoes
    estatisticas_gerais = pd.DataFrame(data = d1)
    
    #Casos os títulos estejam presentes (valor nao e None):
    #vamos utiliza-los
    #Caso contrario, vamos criar nomenclaturas genericas para o histograma
    
    eixo_y = "Counting/Frequency"
    
    if not (data_units_label is None):
        xlabel = data_units_label
    
    else:
        xlabel = "Frequency\n table data"
    
    if not (y_title is None):
        eixo_x = y_title
        #lembre-se que no histograma, os dados originais vao pro eixo X
        #O eixo Y vira o eixo da contagem/frequencia daqueles dados
    
    else:
        eixo_x = "X: Mean value of the interval"
    
    if not (histogram_title is None):
        string1 = "- $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        main_label = histogram_title + string1
        #concatena a string do titulo a string com a media e desvio-padrao
        #%.2f: o numero entre %. e f indica a quantidade de casas decimais da 
        #variavel float f. No caso, arredondamos para 2 casas
        #NAO SE ESQUECA DO PONTO: ele que indicara que sera arredondado o 
        #numero de casas
    
    else:
        main_label = "Data Histogram - $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        #os simbolos $\ $ substituem o simbolo pela letra grega
    
    d2 = {"Considered interval": hist_labels, eixo_x: xhist, eixo_y: yhist}
    #dicionario que compoe a tabela de frequencias
    tab_frequencias = pd.DataFrame(data = d2)
    #cria a tabela de frequencias como um dataframe de saida
   
    #parametros da normal ja calculados:
    #mu e sigma
    #numero de bins: ncells
    #limites de especificacao: lsl,usl - target
    
    #valor maximo do histograma
    max_hist = max(yhist)
    #seleciona o valor maximo da serie, para ajustar a curva normal
    #isso porque a normal é criada com valores entre 0 e 1
    #multiplicando ela por max_hist, fazemos ela se adequar a altura do histograma
    
    if (normal_curve_overlay == True):
        
        #construir a normal ajustada/esperada
        #vamos criar pontos ao redor da media mu - 4sigma ate mu + 4sigma, 
        #de modo a garantir a quase totalidade da curva normal. 
        #O incremento será de 0.10 sigma a cada iteracao
        x_inf = mu -(4)*sigma
        x_sup = mu + 4*sigma
        x_inc = (0.10)*sigma
        
        x_normal_adj = []
        y_normal_adj = []
        
        x_adj = x_inf
        y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
        x_normal_adj.append(x_adj)
        y_normal_adj.append(y_adj)
        
        while(x_adj < x_sup): 
            
            x_adj = x_adj + x_inc
            y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
            x_normal_adj.append(x_adj)
            y_normal_adj.append(y_adj)
        
        #vamos ajustar a altura da curva ao histograma. Para isso, precisamos
        #calcular quantas vezes o ponto mais alto do histograma é maior que o ponto
        #mais alto da normal (chamaremos essa relação de fator). A seguir,
        #multiplicamos cada elemento da normal por este mesmo fator
        max_normal = max(y_normal_adj) 
        #maximo da normal ajustada, numero entre 0 e 1
        
        fator = (max_hist)/(max_normal)
        size_normal = len(y_normal_adj) #quantidade de dados criados
        
        i = 0
        while (i < size_normal):
            y_normal_adj[i] = (y_normal_adj[i])*(fator)
            i = i + 1
    
    #Fazer o grafico
    fig, ax = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    #STANDARD MATPLOTLIB METHOD:
    #bins = number of bins (intervals) of the histogram. Adjust it manually
    #increasing bins will increase the histogram's resolution, but height of bars
    
    #ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #IF GRAPHIC IS NOT SHOWN: THAT IS BECAUSE THE DISTANCES BETWEEN VALUES ARE LOW, AND YOU WILL
    #HAVE TO USE THE STANDARD HISTOGRAM METHOD FROM MATPLOTLIB.
    #TO DO THAT, UNMARK LINE ABOVE: ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #AND MARK LINE BELOW AS COMMENT: ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    
    #IF YOU WANT TO CREATE GRAPHIC AS A BAR CHART BASED ON THE CALCULATED DISTRIBUTION TABLE, 
    #MARK THE LINE ABOVE AS COMMENT AND UNMARK LINE BELOW:
    ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    #ajuste manualmente a largura, width, para deixar as barras mais ou menos proximas
    
    if (normal_curve_overlay == True):
    
        #adicionar a normal
        ax.plot(x_normal_adj, y_normal_adj, color = 'black', label = 'Adjusted/expected\n normal curve')
    
    ax.set_xlabel(eixo_x)
    ax.set_ylabel(eixo_y)
    ax.set_title(main_label)
    ax.set_xticks(xhist)
    
    ax.legend()
    ax.grid(grid)
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "histogram"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    print("General statistics:\n")
    print(estatisticas_gerais)
    print("\n") # line break
    print("Frequency table:\n")
    print(tab_frequencias)

    return estatisticas_gerais, tab_frequencias

In [29]:
def histogram_alternative (y, total_of_bins, bar_width, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, data_units_label = None, y_title = None, histogram_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    
    #Calculo do bin size - largura do histograma:
    #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
    #2: Calcular rangehist = highest - lowest
    #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
    #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
    #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
    #5: Calcular binsize = rangehist/(ncells)
    #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
    #isso porque a largura do histograma tem que ser um numero positivo
    
    #this variable here is to simply guarantee the compatibility of the function,
    # with no extensive code modifications. It has no real effect.
    normal_curve_overlay = True
    

    y = y.reset_index(drop=True)
    #faz com que os indices desta serie sejam consecutivos e a partir de zero

    #Estatisticas gerais: media (mu) e desvio-padrao (sigma)
    mu = y.mean() 
    sigma = y.std() 

    #Calculo do bin-size
    highest = y.max()
    lowest = y.min()
    rangehist = highest - lowest
    rangehist = abs(rangehist)
    #garante que sera um numero positivo
    samplesize = y.count() #contagem do total de entradas
    ncells = (samplesize)**0.5 #potenciacao: ** - raiz quadrada de samplesize
    #resultado da raiz quadrada e sempre positivo
    ncells = round(ncells) #numero "redondo" mais proximo
    ncells = int(ncells) #parte inteira do numero arredondado
    #ncells = numero de linhas da tabela de frequencias
    binsize = rangehist/ncells
    binsize = round(binsize)
    binsize = int(binsize) #precisa ser inteiro
    
    #Construcao da tabela de frequencias

    j = 0 #indice da tabela de frequencias
    #Este indice e diferente do ordenamento dos valores em ordem crescente
    xhist = []
    #Lista vazia que contera os x do histograma
    yhist = []
    #Listas vazia que conteras o y do histograma
    hist_labels = []
    #Esta lista gravara os limites da barra na forma de strings

    pontomediodabarra = lowest + binsize/2 
    limitedabarra = lowest + binsize
    #ponto medio da barra 
    #limite da primeira barra do histograma
    seriedohist1 = y
    seriedohist1 = seriedohist1.sort_values(ascending=True)
    #serie com os valores em ordem crescente
    seriedohist1 = seriedohist1.reset_index(drop=True)
    #garante que a nova serie tenha indices consecutivos, iniciando em zero
    i = 0 #linha inicial da serie do histograma em ordem crescente
    valcomparado = seriedohist1[i]
    #primeiro valor da serie, o mais baixo
        
    estatisticas_col1 = []
    #contera as descricoes das colunas da tabela de estatisticas gerais
    estatisticas_col2 = []
    #contera os valores da tabela de estatisticas gerais
    
    estatisticas_col1.append("Count of data evaluated")
    estatisticas_col2.append(samplesize)
    estatisticas_col1.append("Average (mu)")
    estatisticas_col2.append(mu)
    estatisticas_col1.append("Standard deviation (sigma)")
    estatisticas_col2.append(sigma)
    estatisticas_col1.append("Highest value")
    estatisticas_col2.append(highest)
    estatisticas_col1.append("Lowest value")
    estatisticas_col2.append(lowest)
    estatisticas_col1.append("Data range (maximum value - lowest value)")
    estatisticas_col2.append(rangehist)
    estatisticas_col1.append("Bin size (bar width)")
    estatisticas_col2.append(binsize)
    estatisticas_col1.append("Total rows in frequency table")
    estatisticas_col2.append(ncells)
    #como o comando append grava linha a linha em sequencia, garantimos
    #a correspondencia das colunas
    #Assim como em qualquer string, incluindo de rotulos de graficos
    #os \n sao lidos como quebra de linha
    
    d1 = {"General Statistics": estatisticas_col1, "Calculated Value": estatisticas_col2}
    #dicionario das duas series, para criar o dataframe com as descricoes
    estatisticas_gerais = pd.DataFrame(data = d1)
    
    #Casos os títulos estejam presentes (valor nao e None):
    #vamos utiliza-los
    #Caso contrario, vamos criar nomenclaturas genericas para o histograma
    
    eixo_y = "Counting/Frequency"
    
    if not (data_units_label is None):
        xlabel = data_units_label
    
    else:
        xlabel = "Frequency\n table data"
    
    if not (y_title is None):
        eixo_x = y_title
        #lembre-se que no histograma, os dados originais vao pro eixo X
        #O eixo Y vira o eixo da contagem/frequencia daqueles dados
    
    else:
        eixo_x = "X: Mean value of the interval"
    
    if not (histogram_title is None):
        string1 = "- $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        main_label = histogram_title + string1
        #concatena a string do titulo a string com a media e desvio-padrao
        #%.2f: o numero entre %. e f indica a quantidade de casas decimais da 
        #variavel float f. No caso, arredondamos para 2 casas
        #NAO SE ESQUECA DO PONTO: ele que indicara que sera arredondado o 
        #numero de casas
    
    else:
        main_label = "Data Histogram - $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        #os simbolos $\ $ substituem o simbolo pela letra grega
    
    d2 = {"Considered interval": hist_labels, eixo_x: xhist, eixo_y: yhist}
    #dicionario que compoe a tabela de frequencias
    tab_frequencias = pd.DataFrame(data = d2)
    #cria a tabela de frequencias como um dataframe de saida
    
    #parametros da normal ja calculados:
    #mu e sigma
    #numero de bins: ncells
    #limites de especificacao: lsl,usl - target
    
    #valor maximo do histograma
    #max_hist = max(yhist)
    #seleciona o valor maximo da serie, para ajustar a curva normal
    #isso porque a normal é criada com valores entre 0 e 1
    #multiplicando ela por max_hist, fazemos ela se adequar a altura do histograma
    
    
    #Fazer o grafico
    fig, ax = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    #STANDARD MATPLOTLIB METHOD:
    #bins = number of bins (intervals) of the histogram. Adjust it manually
    #increasing bins will increase the histogram's resolution, but height of bars
    
    ax.hist(y, bins = total_of_bins, width = bar_width, label=xlabel, color='blue')
    #IF GRAPHIC IS NOT SHOWN: THAT IS BECAUSE THE DISTANCES BETWEEN VALUES ARE LOW, AND YOU WILL
    #HAVE TO USE THE STANDARD HISTOGRAM METHOD FROM MATPLOTLIB.
    #TO DO THAT, UNMARK LINE ABOVE: ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #AND MARK LINE BELOW AS COMMENT: ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    
    #IF YOU WANT TO CREATE GRAPHIC AS A BAR CHART BASED ON THE CALCULATED DISTRIBUTION TABLE, 
    #MARK THE LINE ABOVE AS COMMENT AND UNMARK LINE BELOW:
    #ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    #ajuste manualmente a largura, width, para deixar as barras mais ou menos proximas
    
    ax.set_xlabel(eixo_x)
    ax.set_ylabel(eixo_y)
    ax.set_title(main_label)
    #ax.set_xticks(xhist)
    
    ax.legend()
    ax.grid(grid)
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "histogram_alternative"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    print("General statistics:\n")
    print(estatisticas_gerais)
    # This function is supposed to be used in cases where the differences between data
    # is very small. In such cases, there will be no trust values calculated for the 
    # frequency table. Therefore, we omit it here, but it can be accessed from the
    # returned dataframe.

    return estatisticas_gerais, tab_frequencias

# **Function for exporting the dataframe**

In [ ]:
def export_dataframe (dataframe_to_be_exported, new_file_name_with_csv_extension, file_directory_path = None, export_to_s3_bucket = False, s3_bucket_name = None, desired_s3_file_name_with_csv_extension = None):
    
    import os
    import boto3
    #boto3 is AWS S3 Python SDK
    import pandas as pd
    
    ## WARNING: all file extensions should be .csv for this function
    
    # FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
    # (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
    # or FILE_DIRECTORY_PATH = "/folder"
    # If you want to export the file to AWS S3, this parameter will have no effect.
    # In this case, you can set FILE_DIRECTORY_PATH = None

    # NEW_FILE_NAME_WITH_CSV_EXTENSION - (string, in quotes): input the name of the 
    # file with the  extension. e.g. FILE_NAME_WITH_CSV_EXTENSION = "file.csv"
    
    # export_to_s3_bucket = False. Alternatively, set as True to export the file to an
    # AWS S3 Bucket.

    ## The following parameters have effect only when export_to_s3_bucket == True:

    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"

    # The name desired for the object stored in S3 (string, in quotes). 
    # Keep it None to set it equals to new_file_name_with_csv_extension. 
    # Alternatively, set it as a string analogous to new_file_name_with_csv_extension. 
    # e.g. desired_s3_file_name_with_csv_extension = "S3_file.csv"
    
    if (export_to_s3_bucket == True):
        
        if (desired_s3_file_name_with_csv_extension is None):
            #Repeat new_file_name_with_extension
            desired_s3_file_name_with_csv_extension = new_file_name_with_csv_extension
        
        # If the bucket name was provided, start the session. If not, print an error
        # message:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name to download from.")
        
        else:
        
            # start S3 client:
            print("Starting AWS S3 client.")
        
            # Let's export the file to a AWS S3 (simple storage service) bucket
            # instantiate S3 client and upload to s3
            s3_client = boto3.resource('s3')
            
            # Create a local copy of the file on the root.
            local_copy_path = os.path.join("/", new_file_name_with_csv_extension)
            dataframe_to_be_exported.to_csv(local_copy_path, index = False)
            
            print("Local copy of the dataframe created on the root path to export to S3.")
            print("Simply delete this file from the root path if you only want to keep the S3 version.")
            
            # Upload this local copy to S3:
            try:
                response = s3_client.meta.client.upload_file(local_copy_path, s3_bucket_name, desired_s3_file_name_with_extension)
            
            except ClientError as e:
                logging.error(e)
                return False
            
            print(f"{desired_s3_file_name_with_csv_extension} successfully exported to {s3_bucket_name} AWS S3 bucket.")
            return True
            # Check AWS Documentation:
            # https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
            
            # Notice: if you wanted to authenticate directly from Python code, you could use
            # the following code, instead:        
            # ACCESS_KEY = 'access_key_ID'
            # PASSWORD_KEY = 'password_key'
            # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
            # s3_client.upload_file(local_copy_path, s3_bucket_name, desired_s3_file_name_with_extension)
            
    else :
        # Do not export to AWS S3. Export to other path.
        # Create the complete file path:
        file_path = os.path.join(file_directory_path, new_file_name_with_csv_extension)

        dataframe_to_be_exported.to_csv(file_path, index = False)

        print(f"Dataframe {new_file_name_with_csv_extension} exported as \'{file_path}\'.")
        print("Warning: if there was a file in this file path, it was replaced by the exported dataframe.")

# **Function for importing or exporting models and dictionaries**

In [3]:
def import_export_model_or_dict (action = 'import', objects_manipulated = 'model_only', model_file_name = None, dictionary_file_name = None, directory_path = '/', model_type = 'keras', dict_to_export = None, model_to_export = None, use_colab_memory = False):
    
    import os
    import pickel as pkl
    import dill
    import tensorflow as tf
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neural_network import MLPRegressor
    from sklearn.neural_network import MLPClassifier
    from xgboost import XGBRegressor
    from xgboost import XGBClassifier
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.arima.model import ARIMAResults
    from google.colab import files
    
    # action = 'import' for importing a model and/or a dictionary;
    # action = 'export' for exporting a model and/or a dictionary.
    
    # objects_manipulated = 'model_only' if only a model will be manipulated.
    # objects_manipulated = 'dict_only' if only a dictionary will be manipulated.
    # objects_manipulated = 'model_and_dict' if both a model and a dictionary will be
    # manipulated.
    
    #model_file_name: string with the name of the file containing the model (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. model_file_name = 'model'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep model_file_name = None if no model will be manipulated.
    
    # dictionary_file_name: string with the name of the file containing the dictionary 
    # (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. dictionary_file_name = 'history_dict'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no 
    # dictionary will be manipulated.
    
    # DIRECTORY_PATH: path of the directory where the model will be saved,
    # or from which the model will be retrieved. If no value is provided,
    # the DIRECTORY_PATH will be the root: "/"
    # Notice that the model and the dictionary must be stored in the same path.
    # If a model and a dictionary will be exported, they will be stored in the same
    # DIRECTORY_PATH.
    
    # model_type: This parameter has effect only when a model will be manipulated.
    # model_type: 'keras' for deep learning keras/ tensorflow models with extension .h5
    # model_type = 'sklearn' for models from scikit-learn (non-deep learning)
    # model_type = 'xgb_regressor' for XGBoost regression models (non-deep learning)
    # model_type = 'xgb_classifier' for XGBoost classification models (non-deep learning)
    # model_type = 'arima' for ARIMA model (Statsmodels)
    
    # dict_to_export and model_to_export: 
    # These two parameters have effect only when ACTION == 'export'. In this case, they
    # must be declared. If ACTION == 'export', keep:
    # dict_to_export = None, 
    # model_to_export = None
    # If one of these objects will be exported, substitute None by the name of the object
    # e.g. if your model is stored in the global memory as 'keras_model' declare:
    # model_to_export = keras_model. Notice that it must be declared without quotes, since
    # it is not a string, but an object.
    # For exporting a dictionary named as 'dict':
    # dict_to_export = dict
    
    # use_colab_memory: this parameter has only effect when using Google Colab (or it will
    # raise an error). Set as use_colab_memory = True if you want to use the instant memory
    # from Google Colaboratory: you will update or download the file and it will be available
    # only during the time when the kernel is running. It will be excluded when the kernel
    # dies, for instance, when you close the notebook.
    
    # If action == 'export' and use_colab_memory == True, then the file will be downloaded
    # to your computer (running the cell will start the download).
    
    # Check the directory path
    if (directory_path is None):
        # set as the root:
        directory_path = "/"
        
        
    bool_check1 = (objects_manipulated != 'model_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    bool_check2 = (objects_manipulated != 'dict_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    if (bool_check1 == True):
        #manipulate a dictionary
        
        if (dictionary_file_name is None):
            print("Please, enter a name for the dictionary.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            dict_path = os.path.join(directory_path, dictionary_file_name)
            # Extract the file extension
            dict_extension = 'pkl'
            #concatenate:
            dict_path = dict_path + "." + dict_extension
            
    
    if (bool_check2 == True):
        #manipulate a model
        
        if (model_file_name is None):
            print("Please, enter a name for the model.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            model_path = os.path.join(directory_path, model_file_name)
            # Extract the file extension
            
            #check model_type:
            if (model_type == 'keras'):
                model_extension = 'h5'
            
            elif (model_type == 'sklearn'):
                model_extension = 'dill'
                #it could be 'pkl', though
            
            elif (model_type == 'xgb_regressor'):
                model_extension = 'json'
                #it could be 'ubj', though
            
            elif (model_type == 'xgb_classifier'):
                model_extension = 'json'
                #it could be 'ubj', though
            
            elif (model_tyoe == 'arima'):
                model_extension = 'pkl'
            
            else:
                print("Enter a valid model_type: keras, sklearn_xgb, or arima.")
                return "error2"
            
            #concatenate:
            model_path = model_path +  "." + model_extension
            
    # Now we have the full paths for the dictionary and for the model.
    
    if (action == 'import'):
        
        if (use_colab_memory == True):
            
            print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
            print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
            # this functionality requires the previous declaration:
            ## from google.colab import files
            colab_files_dict = files.upload()
            # The files are stored into a dictionary called colab_files_dict where the keys
            # are the names of the files and the values are the files themselves.
            ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
            ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
            ## representing the contents of the file. The length of this value is the size of the
            ## uploaded file, in bytes.
            ## To access the file is like accessing a value from a dictionary: 
            ## d = {'key1': 'val1'}, d['key1'] == 'val1'
            ## we simply declare the key inside brackets and quotes, the same way we would do for
            ## accessing the column of a dataframe.
            ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
            ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
            ## file in bytes.
            ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
            ## parentheses): colab_files_dict.keys()
            
            for key in colab_files_dict.keys():
                #loop through each element of the list of keys of the dictionary
                # (list colab_files_dict.keys()). Each element is named 'key'
                print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
                # The key is the name of the file, and the length of the value
                ## correspondent to the key is the file's size in bytes.
                ## Notice that the content of the uploaded object must be passed 
                ## as argument for a proper function to be interpreted. 
                ## For instance, the content of a xlsx file should be passed as
                ## argument for Pandas .read_excel function; the pkl file must be passed as
                ## argument for pickle.
                ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
                ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
                ## df from the uploaded table. Notice that is the value, not the key, that is the
                ## argument.
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                key = dictionary_file_name + "." + dict_extension
                #Use the key to access the file content, and pass the file content
                # to pickle:
                with open(colab_files_dict[key], 'rb') as opened_file:
            
                    imported_dict = pkl.load(opened_file)
                    # The structure imported_dict = pkl.load(open(colab_files_dict[key], 'rb')) relies 
                    # on the GC to close the file. That's not a good idea: If someone doesn't use 
                    # CPython the garbage collector might not be using refcounting (which collects 
                    # unreferenced objects immediately) but e.g. collect garbage only after some time.
                    # Since file handles are closed when the associated object is garbage collected or 
                    # closed explicitly (.close() or .__exit__() from a context manager) the file 
                    # will remain open until the GC kicks in.
                    # Using 'with' ensures the file is closed as soon as the block is left - even if 
                    # an exception happens inside that block, so it should always be preferred for any 
                    # real application.
                    # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python

                print(f"Dictionary {key} successfully imported to Colab environment.")
            
            else:
                #standard method
                with open(dict_path, 'rb') as opened_file:
            
                    imported_dict = pkl.load(opened_file)
                
                # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'
                print(f"Dictionary successfully imported from {dict_path}.")
                
        if (bool_chek2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = tf.keras.models.load_model(colab_files_dict[key])
                    print(f"Keras/TensorFlow model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from keras.models import load_model
                    model = tf.keras.models.load_model(model_path)
                    print(f"Keras/TensorFlow model successfully imported from {model_path}.")

            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    
                    with open(colab_files_dict[key], 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                    
                    print(f"Scikit-learn model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                
                    print(f"Scikit-learn model successfully imported from {model_path}.")
                    # For loading a pickle model:
                    ## model = pkl.load(open(model_path, 'rb'))
                    # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'

            elif (model_type == 'xgb_regressor'):
                
                # Create an instance (object) from the class XGBRegressor:
                
                model = XGBRegressor()
                # Now we can apply the load_model method from this class:
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = model.load_model(colab_files_dict[key])
                    print(f"XGBoost regression model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    model = model.load_model(model_path)
                    print(f"XGBoost regression model successfully imported from {model_path}.")
                    # model.load_model("model.json") or model.load_model("model.ubj")
                    # .load_model is a method from xgboost object
            
             elif (model_type == 'xgb_classifier'):
                
                # Create an instance (object) from the class XGBClassifier:
                
                model = XGBClassifier()
                # Now we can apply the load_model method from this class:
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = model.load_model(colab_files_dict[key])
                    print(f"XGBoost classification model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    model = model.load_model(model_path)
                    print(f"XGBoost classification model successfully imported from {model_path}.")
                    # model.load_model("model.json") or model.load_model("model.ubj")
                    # .load_model is a method from xgboost object

            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = ARIMAResults.load(colab_files_dict[key])
                    print(f"ARIMA model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from statsmodels.tsa.arima.model import ARIMAResults
                    model = ARIMAResults.load(model_path)
                    print(f"ARIMA model successfully imported from {model_path}.")
            
            if (objects_manipulated == 'model_only'):
                # only the model should be returned
                return model
            
            elif (objects_manipulated == 'dict_only'):
                # only the dictionary should be returned:
                return imported_dict
            
            else:
                # Both objects are returned:
                return model, imported_dict

    
    elif (action == 'export'):
        
        #Let's export the models or dictionary:
        if (use_colab_memory == True):
            print("The files will be downloaded to your computer.")
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                ## Download the dictionary
                key = dictionary_file_name + "." + dict_extension
                
                with open(key, 'wb') as opened_file:
            
                    pkl.dump(dict_to_export, opened_file)
                
                # this functionality requires the previous declaration:
                ## from google.colab import files
                files.download(key)
                
                print(f"Dictionary {key} successfully downloaded from Colab environment.")
            
            else:
                #standard method 
                with open(dict_path, 'wb') as opened_file:
            
                    pkl.dump(dict_to_export, opened_file)
                
                #to save the file, the mode must be set as 'wb' (write binary)
                print(f"Dictionary successfully exported as {dict_path}.")
                
        if (bool_chek2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"Keras/TensorFlow model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"Keras/TensorFlow model successfully exported as {model_path}.")

            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    
                    with open(key, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    #to save the file, the mode must be set as 'wb' (write binary)
                    files.download(key)
                    print(f"Scikit-learn model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    print(f"Scikit-learn model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif ((model_type == 'xgb_regressor')|(model_type == 'xgb_classifier')):
                # In both cases, the XGBoost object is already loaded in global
                # context memory. So there is already the object for using the
                # save_model method, available for both classes (XGBRegressor and
                # XGBClassifier).
                # We can simply check if it is one type OR the other, since the
                # method is the same:
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save_model(key)
                    files.download(key)
                    print(f"XGBoost model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save_model(model_path)
                    print(f"XGBoost model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"ARIMA model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"ARIMA model successfully exported as {model_path}.")
        
        print("Export of files completed.")
    
    else:
        print("Enter a valid action, import or export.")

# **Function for downloading a file from Google Colab or AWS S3 to the local machine or uploading a file from the machine to S3 or to Colab's instant memory**

In [2]:
def download_or_upload_file (source = 'aws', action = 'download', object_to_download_from_colab = None, s3_bucket_name = None, local_path_of_storage = '/', file_name_with_extension = None):
    
    import os
    import boto3
    # boto3 is AWS S3 Python SDK
    from google.colab import files
    
    # source = 'google' for downloading from (or uploading to) Google Colab's instant memory;
    # source = 'aws' for downloading from (or uploading to) an AWS S3 bucket.
    
    # action = 'download' to download the file to the local machine
    # action = 'upload' to upload a file from local machine to AWS S3 or to
    # Google Colab's instant memory
    
    # object_to_download_from_colab = None. This option has effect only when
    # source == 'google'. In this case, this parameter is obbligatory. 
    # Declare as object_to_download_from_colab the object that you want to download.
    # Since it is an object and not a string, it should not be declared in quotes.
    # e.g. to download a dictionary named dict, object_to_download_from_colab = dict.
    # To download a dataframe named df, declare object_to_download_from_colab = df.
    # To export a model named keras_model, declare object_to_download_from_colab = keras_model
    
    ## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # LOCAL_PATH_OF_STORAGE: path of the local computer environment 
    # to which the S3 bucket contents will be downloaded (ACTION == 'download'); or
    # path of the folder containing the file that will be uploaded in S3 (ACTION = 'upload'). 
    # If it is None, or if LOCAL_PATH_OF_STORAGE = '/', files 
    # will be imported to the root path. Alternatively, input the path as a string 
    # (in quotes).
    # Examples: LOCAL_PATH_OF_STORAGE = '/copied_s3_bucket'; 
    # LOCAL_PATH_OF_STORAGE = "/My_folder"; LOCAL_PATH_OF_STORAGE = "/Users/Me/Documents/"
    # Notice that only the directories should be declared: do not include the file name and
    # its extension.
    
    # file_name_with_extension: string, in quotes, containing the file name which will be
    # downloaded from S3; or uploaded from S3, followed by its extension. 
    ## This parameter is obbligatory when source == 'aws'
    # Examples:
    # file_name_with_extension = 'Screen_Shot.png'; file_name_with_extension = 'dataset.csv',
    # file_name_with_extension = "dictionary.pkl", file_name_with_extension = "model.h5",
    # file_name_with_extension = 'doc.pdf', file_name_with_extension = 'model.dill'

    if (source == 'google'):
        
        if (action == 'upload'):
            
            print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
            print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
            # this functionality requires the previous declaration:
            ## from google.colab import files
            
            colab_files_dict = files.upload()
            
            # The files are stored into a dictionary called colab_files_dict where the keys
            # are the names of the files and the values are the files themselves.
            ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
            ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
            ## representing the contents of the file. The length of this value is the size of the
            ## uploaded file, in bytes.
            ## To access the file is like accessing a value from a dictionary: 
            ## d = {'key1': 'val1'}, d['key1'] == 'val1'
            ## we simply declare the key inside brackets and quotes, the same way we would do for
            ## accessing the column of a dataframe.
            ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
            ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
            ## file in bytes.
            ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
            ## parentheses): colab_files_dict.keys()
            
            for key in colab_files_dict.keys():
                #loop through each element of the list of keys of the dictionary
                # (list colab_files_dict.keys()). Each element is named 'key'
                print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
                # The key is the name of the file, and the length of the value
                ## correspondent to the key is the file's size in bytes.
                ## Notice that the content of the uploaded object must be passed 
                ## as argument for a proper function to be interpreted. 
                ## For instance, the content of a xlsx file should be passed as
                ## argument for Pandas .read_excel function; the pkl file must be passed as
                ## argument for pickle.
                ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
                ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
                ## df from the uploaded table. Notice that is the value, not the key, that is the
                ## argument.
                
                print("The uploaded files are stored into a dictionary object named as colab_files_dict.")
                print("Each key from this dictionary is the name of an uploaded file. The value correspondent to that key is the file itself.")
                print("The structure of a general Python dictionary is dict = {\'key1\': value1}. To access value1, declare file = dict[\'key1\'], as if you were accessing a column from a dataframe.")
                print("Then, if you uploaded a file named \'table.xlsx\', you can access this file as:")
                print("uploaded_file = colab_files_dict[\'table.xlsx\']")
                print("Notice, though, that the object uploaded_file is the whole file content, not a Python object already converted. To convert to a Python object, pass this element as argument for a proper function or method.")
                print("In this example, to convert the object uploaded_file to a dataframe, Pandas pd.read_excel function could be used. In the following line, a df dataframe object is obtained from the uploaded file:")
                print("df = pd.read_excel(uploaded_file)")
        
        elif (action == 'download'):
            
            if (object_to_download_from_colab is None):
                
                #No object was declared
                print("Please, inform an object to download. Since it is an object, not a string, it should not be declared in quotes.")
            
            else:
                
                print("The file will be downloaded to your computer.")

                files.download(object_to_download_from_colab)

                print(f"File {object_to_download_from_colab} successfully downloaded from Colab environment.")

        else:
            
            print("Please, select a valid action, download or upload.")
          
    elif (source == 'aws'):
        
        # Notice: if you wanted to authenticate directly from Python code, you could use
        # the following code, instead for starting the client:
        
        # ACCESS_KEY = 'access_key_ID'
        # PASSWORD_KEY = 'password_key'
        # s3_client = boto3.client('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = PASSWORD_KEY)
        # Nextly, the code is the same.
        
        
        # If the path to store is None, also import the bucket content to root path;
        # or upload the file from root path to the bucket
        if (local_path_of_storage is None):
            
            local_path_of_storage = '/'
        
        # If the bucket name was provided, start the session. If not, print an error
        # message. The same for the file name with extension:
        
        if (s3_bucket_name is None):
            
            print("Please, provide a valid S3 Bucket name.")
        
        elif (file_name_with_extension is None):
            
            print("Please, provide a valid file name with its extension. e.g. \'dataset.csv\'.")
        
        else:
            
            # Obtain the full file path from which the file will be uploaded to S3; or to
            # which the file will be downloaded from S3:
            file_path = os.path.join(local_path_of_storage, file_name_with_extension)
            
            # Start S3 client:
            s3_client = boto3.resource('s3')
            
            print("Starting AWS S3 client.")
            
            if (action == 'upload'):
                
                s3_client.Object(s3_bucket_name, file_name_with_extension).\
                    upload_file(Filename = file_path)
                
                print(f"File {file_name_with_extension} successfully uploaded to AWS S3 {s3_bucket_name} bucket.")
            
            elif (action == 'download'):

                print("The file will be downloaded to your computer.")
                
                s3_client.Object(s3_bucket_name, file_name_with_extension).download_file(file_path)
                
                print(f"File {file_name_with_extension} successfully downloaded from AWS S3 {s3_bucket_name} bucket.")

            else:

                print("Please, select a valid action, download or upload.")

    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

## **Call the functions**

### **Mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for mounting the google drive;
# SOURCE = 'aws' for accessing an AWS S3 bucket

## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN SOURCE == 'aws':

PATH_TO_STORE_IMPORTED_S3_BUCKET = '/'
# PATH_TO_STORE_IMPORTED_S3_BUCKET: path of the Python environment to which the
# S3 bucket contents will be imported. If it is None, or if 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/', bucket will be imported to the root path. 
# Alternatively, input the path as a string (in quotes). e.g. 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/copied_s3_bucket'

S3_BUCKET_NAME = 'name_of_aws_s3_bucket_to_be_accessed'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_KEY_PREFFIX_FOLDER = None
# S3_OBJECT_KEY_PREFFIX_FOLDER = None. Keep it None or as an empty string 
# (S3_OBJECT_KEY_PREFFIX_FOLDER = '') to import the whole bucket content, instead of a 
# single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, key_preffix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

mount_storage_system (source = SOURCE, path_to_store_imported_s3_bucket = PATH_TO_STORE_IMPORTED_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, s3_obj_key_preffix = S3_OBJECT_KEY_PREFFIX_FOLDER)

### **Importing the dataset**

In [3]:
## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, etc), 
## JSON, txt, or CSV (comma separated values) files.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"

FILE_NAME_WITH_EXTENSION = "dataset.csv"
# FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
# FILE_NAME_WITH_EXTENSION = "file.csv"

LOAD_TXT_FILE_WITH_JSON_FORMAT = False
# LOAD_TXT_FILE_WITH_JSON_FORMAT = False. Set LOAD_TXT_FILE_WITH_JSON_FORMAT = True 
# if you want to read a file with txt extension containing a text formatted as JSON 
# (but not saved as JSON).
# WARNING: if LOAD_TXT_FILE_WITH_JSON_FORMAT = True, all the JSON file parameters of the 
# function (below) must be set. If not, an error message will be raised.
    
HAS_HEADER = True
# HAS_HEADER = True if the the imported table has headers (row with columns names).
# Alternatively, HAS_HEADER = False if the dataframe does not have header.

## Parameters for loading txt or CSV files:

TXT_CSV_COL_SEP = "comma"
# TXT_CSV_COL_SEP = "comma" - This parameter has effect only when the file is a 'txt'
# or 'csv'. It informs how the different columns are separated.
# Alternatively, TXT_CSV_COL_SEP = "comma" for columns separated by comma (",")
# TXT_CSV_COL_SEP = "whitespace" for columns separated by simple spaces (" ").

SHEET_TO_LOAD = None
# SHEET_TO_LOAD - This parameter has effect only when for Excel files.
# keep SHEET_TO_LOAD = None not to specify a sheet of the file, so that the first sheet
# will be loaded.
# SHEET_TO_LOAD may be an integer or an string (inside quotes). SHEET_TO_LOAD = 0
# loads the first sheet (sheet with index 0); SHEET_TO_LOAD = 1 loads the second sheet
# of the file (index 1); SHEET_TO_LOAD = "Sheet1" loads a sheet named as "Sheet1".
# Declare a number to load the sheet with that index, starting from 0; or declare a
# name to load the sheet with that name.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFFIX_LIST = None
# JSON_METADATA_PREFFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFFIX_LIST = ['name', 'last']

#The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = load_dataframe (file_directory_path = FILE_DIRECTORY_PATH, file_name_with_extension = FILE_NAME_WITH_EXTENSION, load_txt_file_with_json_format = LOAD_TXT_FILE_WITH_JSON_FORMAT, has_header = HAS_HEADER, txt_csv_col_sep = TXT_CSV_COL_SEP, sheet_to_load = SHEET_TO_LOAD, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_preffix_list = JSON_METADATA_PREFFIX_LIST)

### **Converting JSON object to dataframe**

In [3]:
JSON_OBJ_TO_CONVERT = json_object #Alternatively: object containing the JSON to be converted

# JSON_OBJ_TO_CONVERT: object containing JSON, or string with JSON content to parse.
# Objects may be: string with JSON formatted text;
# list with nested dictionaries (JSON formatted);
# dictionaries, possibly with nested dictionaries (JSON formatted).

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFFIX_LIST = None
# JSON_METADATA_PREFFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFFIX_LIST = ['name', 'last']

#The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = json_obj_to_dataframe (json_obj_to_convert = JSON_OBJ_TO_CONVERT, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_preffix_list = JSON_METADATA_PREFFIX_LIST)

### **Filtering (selecting) or renaming columns of the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

MODE = 'filter'
# MODE = 'filter' for filtering only the list of columns passed as cols_list;
# MODE = 'rename' for renaming the columns with the names passed as cols_list.

COLS_LIST = ['column1', 'column2', 'column3']
# COLS_LIST = list of strings containing the names (headers) of the columns to select
# (filter); or to be set as the new columns' names, according to the selected mode.
# For instance: COLS_LIST = ['col1', 'col2', 'col3'] will 
# select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
# Declare the names inside quotes.
# Simply substitute the list by the list of columns that you want to select; or the
# list of the new names you want to give to the dataset columns.

#New dataframe saved as new_df. Simply modify this object on the left of equality:
new_df = col_filter_rename (df = DATASET, cols_list = COLS_LIST, mode = MODE)

### **Calculating general statistics from a given column**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_TO_ANALYZE = 'column_to_analyze'
# COLUMN_TO_ANALYZE = string (inside quotes) containing the name of the column that will be analyzed. 
# e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.

# Statistics dataframe saved as general_stats. 
# Simply modify this object on the left of equality:
general_stats = GENERAL_STATISTICS (df = DATASET, column_to_analyze = COLUMN_TO_ANALYZE)

### **Getting data quantiles from a given column**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_TO_ANALYZE = 'column_to_analyze'
# COLUMN_TO_ANALYZE = string (inside quotes) containing the name of the column that will be analyzed. 
# e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.

# Quantiles dataframe saved as quantiles_summ_df
# Simply modify this object on the left of equality:
quantiles_summ_df = GET_QUANTILES (df = DATASET, column_to_analyze = COLUMN_TO_ANALYZE)

### **Getting a particular P-percent quantile limit (P is a percent, from 0 to 100)**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_TO_ANALYZE = 'column_to_analyze'
# COLUMN_TO_ANALYZE = string (inside quotes) containing the name of the column that will be analyzed. 
# e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.

P_PERCENT = 100
# P_PERCENT: float value from 0 to 100 representing the percent of the quantile
# if P_PERCENT = 31.2, then 31.2% of the data will fall below the returned value
# if P_PERCENT = 75, then 75% of the data will fall below the returned value
# if P_PERCENT = 0, the minimum value is returned.
# if P_PERCENT = 100, the maximum value is returned.

# P-Percent quantile limit returned as the float value quantile_lim
# Simply modify this variable on the left of equality:
quantile_lim = GET_P_PERCENT_QUANTILE_LIM (df = DATASET, column_to_analyze = COLUMN_TO_ANALYZE, p_percent = P_PERCENT)

### **Applying a list of row filters to a dataframe**

In [ ]:
# Warning: this function filter the rows and results into a smaller dataset, 
# since it removes the non-selected entries.
# If you want to pass a filter to simply label the selected rows, use the function 
# LABEL_DATAFRAME_SUBSETS, which do not eliminate entries from the dataframe.
    
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

## define the filters and only them define the filters list
# EXAMPLES OF BOOLEAN FILTERS TO COMPOSE THE LIST
boolean_filter1 = ((None) & (None)) # (condition1 AND (&) condition2)
boolean_filter2 = ((None) | (None)) # condition1 OR (|) condition2
# boolean filters result into boolean values True or False.

## Examples of filters:
## filter1 = (condition 1) & (condition 2)
## filter1 = (df['column1'] > = 0) & (df['column2']) < 0)
## filter2 = (condition)
## filter2 = (df['column3'] <= 2.5)
## filter3 = (df['column4'] > 10.7)
## filter3 = (condition 1) | (condition 2)
## filter3 = (df['column5'] != 'string1') | (df['column5'] == 'string2')
    
## comparative operators: > (higher); >= (higher or equal); < (lower); 
## <= (lower or equal); == (equal); != (different)
    
## concatenation operators: & (and): the filter is True only if the 
## two conditions concatenated through & are True
## | (or): the filter is True if at least one of the two conditions concatenated
## through | are True.
    
## separate conditions with parentheses. Use parentheses to define a order
## of definition of the conditions:
## filter = ((condition1) & (condition2)) | (condition3)
## Here, firstly ((condition1) & (condition2)) = subfilter is evaluated. 
## Then, the resultant (subfilter) | (condition3) is evaluated.

## Pandas .isin method: you can also use this method to filter rows belonging to
## a given subset (the row that is in the subset is selected). The syntax is:
## is_black_or_brown = dogs["color"].isin(["Black", "Brown"])
## or: filter = (dataframe_column_series).isin([value1, value2, ...])

LIST_OF_ROW_FILTERS = [boolean_filter1, boolean_filter2]
# LIST_OF_ROW_FILTERS: list of boolean filters to be applied to the dataframe
# e.g. LIST_OF_ROW_FILTERS = [filter1]
# applies a single filter saved as filter 1. Notice: even if there is a single
# boolean filter, it must be declared inside brackets, as a single-element list.
# That is because the function will loop through the list of filters.
# LIST_OF_ROW_FILTERS = [filter1, filter2, filter3, filter4]
# will apply, in sequence, 4 filters: filter1, filter2, filter3, and filter4.
# Notice that the filters must be declared in the order you want to apply them.

# Filtered dataframe saved as filtered_df
# Simply modify this object on the left of equality:
filtered_df = APPLY_ROW_FILTERS_LIST (df = DATASET, list_of_row_filters = LIST_OF_ROW_FILTERS)

### **Selecting subsets from a dataframe (using row filters) and labelling these subsets**

In [ ]:
# Attention: this function selects subsets from the dataframe and label them, 
# allowing the seggregation of the data.
# If you want to filter the dataframe to eliminate non-selected rows, use the 
# function APPLY_ROW_FILTERS_LIST
    
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

## define the filters and only them define the filters list
# EXAMPLES OF BOOLEAN FILTERS TO COMPOSE THE LIST
boolean_filter1 = ((None) & (None)) # (condition1 AND (&) condition2)
boolean_filter2 = ((None) | (None)) # condition1 OR (|) condition2
# boolean filters result into boolean values True or False.

## Examples of filters:
## filter1 = (condition 1) & (condition 2)
## filter1 = (df['column1'] > = 0) & (df['column2']) < 0)
## filter2 = (condition)
## filter2 = (df['column3'] <= 2.5)
## filter3 = (df['column4'] > 10.7)
## filter3 = (condition 1) | (condition 2)
## filter3 = (df['column5'] != 'string1') | (df['column5'] == 'string2')
    
## comparative operators: > (higher); >= (higher or equal); < (lower); 
## <= (lower or equal); == (equal); != (different)
    
## concatenation operators: & (and): the filter is True only if the 
## two conditions concatenated through & are True
## | (or): the filter is True if at least one of the two conditions concatenated
## through | are True.
    
## separate conditions with parentheses. Use parentheses to define a order
## of definition of the conditions:
## filter = ((condition1) & (condition2)) | (condition3)
## Here, firstly ((condition1) & (condition2)) = subfilter is evaluated. 
## Then, the resultant (subfilter) | (condition3) is evaluated.

## Pandas .isin method: you can also use this method to filter rows belonging to
## a given subset (the row that is in the subset is selected). The syntax is:
## is_black_or_brown = dogs["color"].isin(["Black", "Brown"])
## or: filter = (dataframe_column_series).isin([value1, value2, ...])

LIST_OF_ROW_FILTERS = [boolean_filter1, boolean_filter2]
# LIST_OF_ROW_FILTERS: list of boolean filters to be applied to the dataframe
# e.g. list_of_row_filters = [filter1]
# applies a single filter saved as filter 1. Notice: even if there is a single
# boolean filter, it must be declared inside brackets, as a single-element list.
# That is because the function will loop through the list of filters.
# LIST_OF_ROW_FILTERS = [filter1, filter2, filter3, filter4]
# will apply, in sequence, 4 filters: filter1, filter2, filter3, and filter4.
# Notice that the filters must be declared in the order you want to apply them.

NEW_LABELS_LIST = None
# Here, each filter will be associated with one label. Therefore, if you want to apply 
# a filter with several conditions, combine them using the & and | operators.
    
# The labels are input as a list of strings or of discrete integer values. There must be
# exactly one filter to each label: the filter will be used to obtain a subset from
# the dataframe, and the label will be applied to all the rows of this subset.
# e.g. LIST_OF_ROW_FILTERS = [filter1, filter2], NEW_LABELS_LIST = ['label1', 'label2']
# In this case, the rows of the dataset obtained when applying filter1 will receive the
# string 'label1' as label. In turns, rows selected from filter2 will be labelled as 
# 'label2'.
# LIST_OF_ROW_FILTERS = [filter1, filter2, filter3], NEW_LABELS_LIST = [1, 2, 3]
# Here, the labels are integers: rows filtered through filter1 are marked as 1;
# rows selected by filter2 are labelled as 2; and rows selected from filter3 are marked
# as 3.
    
# Warning: the sequence of filters must be correspondent to the sequence of labels. 
# Rows selected from the first filter are labelled with the first item from the labels
# list; rows selected by the 2nd filter are labelled with the 2nd element, and so on.
    
# If no list of labels is provided, a list of integers starting from zero and with the
# same total of elements from LIST_OF_ROW_FILTERS will be created. For instance,
# if lLIST_OF_ROW_FILTERS = [filter1, filter2, filter3, filter4], the list of labels
# [0, 1, 2, 3] would be created. Again, the rows selected from filter1 would be labelled
# as 0; rows selected from filter 2 would be labelled as 1; and so on.

NEW_LABELLED_COLUMN_NAME = None
# NEW_LABELLED_COLUMN_NAME = None. Alternatively, declare a string (inside quotes), with
# the name of the column containing the applied labels. For example, if
# NEW_LABELLED_COLUMN_NAME = 'labels', then the new labels are saved in the column 'labels'
# If no NEW_LABELLED_COLUMN_NAME is provided, the default name = "df_label" is provided.

# Labelled dataframe saved as labelled_df
# Simply modify this object on the left of equality:
labelled_df = LABEL_DATAFRAME_SUBSETS (df = DATASET, list_of_row_filters = LIST_OF_ROW_FILTERS, new_labels_list = NEW_LABELS_LIST, new_labelled_column_name = NEW_LABELLED_COLUMN_NAME)

### **Performing Analysis of Variance (ANOVA) and obtaining boxplots**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_TO_ANALYZE = 'column_to_analyze'
# COLUMN_TO_ANALYZE = string (inside quotes) containing the name of the column that will be analyzed. 
# e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.

COLUMN_WITH_GROUP_LABELS = None
# COLUMN_WITH_GROUP_LABELS: name of the column registering groups, labels, or
# factors correspondent to each row. e.g. COLUMN_WITH_GROUP_LABELS = "group"
# if the correspondent group is registered in a column named 'group'.
# If no name is provided, then the default name is used:
# COLUMN_WITH_GROUP_LABELS = COLUMN_TO_ANALYZE + "_label"

CONFIDENCE_LEVEL = 0.95
# CONFIDENCE_LEVEL = 0.95 = 95% confidence
# Set CONFIDENCE_LEVEL = 0.90 to get 0.90 = 90% confidence in the analysis.
# Notice that, when less trust is needed, we can reduce CONFIDENCE_LEVEL 
# to get less restrictive results.

REFERENCE_VALUE = None
# reference_value: keep it as None or add a float value.
# This reference value will be shown as a red constant line on the plot to be compared
# with the boxes. e.g. reference_value = 1.0 will plot a red line passing through
# column_to_analyze = 1.0
BOXPLOT_NOTCH = False
# Manipulate parameter notch (boolean, default: False) from the boxplot object
# Whether to draw a notched boxplot (True), or a rectangular boxplot (False). 
# The notches represent the confidence interval (CI) around the median. 
BOXPLOT_VERTICALLY_ORIENTED = True
# Manipulate the parameter vert (boolean, default: True)
# If True, draws vertical boxes. If False, draw horizontal boxes.
BOXPLOT_PATCH_ARTIST = False 
# Manipulate parameter patch_artist (boolean, default: False)
# If False produces boxes with the Line2D artist. Otherwise, boxes are drawn 
# with Patch artists.
X_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.
DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"
FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.
PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

# Dictionary storing ANOVA F-test and p-value returned as anova_summary_dict
# Dictionary mapping each component of the boxplot returned as boxplot_returned_dict
# Simply modify these objects on the left of equality:
anova_summary_dict, boxplot_returned_dict = anova_boxplot (df = DATASET, column_to_analyze = COLUMN_TO_ANALYZE, column_with_group_labels = COLUMN_WITH_GROUP_LABELS, confidence_level = CONFIDENCE_LEVEL, boxplot_notch = BOXPLOT_NOTCH, boxplot_vertically_oriented = BOXPLOT_VERTICALLY_ORIENTED, boxplot_patch_artist = BOXPLOT_PATCH_ARTIST,  reference_value = REFERENCE_VALUE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Concatenating (SQL UNION) multiple dataframes**

In [ ]:
LIST_OF_DATAFRAMES = [dataset1, dataset2]
# LIST_OF_DATAFRAMES must be a list containing the dataframe objects
# example: list_of_dataframes = [df1, df2, df3, df4]
# Notice that the dataframes are objects, not strings. Therefore, they should not
# be declared inside quotes.
# There is no limit of dataframes. In this example, we will concatenate 4 dataframes.
# If LIST_OF_DATAFRAMES = [df1, df2, df3] we would concatenate 3, and if
# LIST_OF_DATAFRAMES = [df1, df2, df3, df4, df5] we would concatenate 5 dataframes.

WHAT_TO_APPEND = 'rows'
# WHAT_TO_APPEND = 'rows' for appending the rows from one dataframe
# into the other; WHAT_TO_APPEND = 'columns' for appending the columns
# from one dataframe into the other (horizontal or lateral append).

IGNORE_INDEX_ON_UNION = True # Alternatively: True or False

SORT_VALUES_ON_UNION = True # Alternatively: True or False

UNION_JOIN_TYPE = None
# JOIN can be 'inner' to perform an inner join, eliminating the missing values
# The default (None) is 'outer': the dataframes will be stacked on the columns with
# same names but, in case there is no correspondence, the row will present a missing
# value for the columns which are not present in one of the dataframes.
# When using the 'inner' method, only the common columns will remain.
# Alternatively, keep UNION_JOIN_TYPE = None for the standard outer join; or set
# UNION_JOIN_TYPE = "inner" (inside quotes) for using the inner join.
    
#These 3 last parameters are the same from Pandas .concat method:
# IGNORE_INDEX_ON_UNION = ignore_index;
# SORT_VALUES_ON_UNION = sort
# UNION_JOIN_TYPE = join
# Check Datacamp course Joining Data with pandas, Chap.3, 
# Advanced Merging and Concatenating
    

#New dataframe saved as concat_df. Simply modify this object on the left of equality:
concat_df = UNION_DATAFRAMES (list_of_dataframes = LIST_OF_DATAFRAMES, what_to_append = WHAT_TO_APPEND, ignore_index_on_union = IGNORE_INDEX_ON_UNION, sort_values_on_union = SORT_VALUES_ON_UNION, union_join_type = UNION_JOIN_TYPE)

### **Importing or exporting models and dictionaries**

#### Case 1: import only a model

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'keras'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

#Model object saved as model.
# Simply modify this object on the left of equality:
model = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 2: import only a dictionary

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'dict_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'keras'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Dictionary saved as imported_dict.
# Simply modify this object on the left of equality:
imported_dict = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 3: import a model and a dictionary

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_and_dict'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'keras'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Model object saved as model. Dictionary saved as imported_dict.
# Simply modify these objects on the left of equality:
model, imported_dict = import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

#### Case 4: export a model and/or a dictionary

In [ ]:
ACTION = 'export'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_FILE_NAME = None
# DICTIONARY_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_FILE_NAME = None if no dictionary will be manipulated.

DIRECTORY_PATH = '/'
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: "/"
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'keras'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

import_export_model_or_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_file_name = DICTIONARY_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_to_export = DICT_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY)    

### **Characterizing the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#New dataframes saved as df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values.
# Simply modify this object on the left of equality:
df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values = df_gen_charac (df = DATASET)

### **Visualizing time series**
        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#X1 = dataset.index to use the index as the axis itself
X1 = (DATASET['DATE']).astype('datetime64[D]') 
#Alternatively: None; or other column in quotes, substituting 'DATE'
# WARNING: Modify only the object in the first parenthesis: DATASET['DATE']
# Do not modify the method .astype('datetime64[D]')
#Remove .astype('datetime64[D]') if it is not a datetime.
# e.g. X1 = DATASET['Time'] for a X variable named 'Time', if 'Time' is a float, not a
# a datetime64. If 'Time' should be interpreted as a timestamp, then, we would declare as:

# X1 = (DATASET['Time']).astype('datetime64[D]')

# In summary: apply the method .astype('datetime64[D]') if you want the value to be
# interpreted (correctly) as a timestamp.

#Notice that there is a data transforming step to guarantee that the 'DATE' was interpreted as a timestamp, not as object or string.
#The astype method defines the type of variable as 'datetime64[D]'. If we wanted the timestamps to be resolved in seconds, we should use
# 'datetime64[ns]'.
Y1 = DATASET['Y1'] 
#Alternatively: None; or other column in quotes, substituting 'Y1'
# e.g. Y1 = DATASET['Speed'] for a Y variable named 'Speed'

X2 = None #Alternatively: series for X2 (analogous to X1)
Y2 = None #Alternatively: series for Y2 (analogous to Y1)
X3 = None #Alternatively: series for X3 (analogous to X1)
Y3 = None #Alternatively: series for Y3 (analogous to Y1)
X4 = None #Alternatively: series for X4 (analogous to X1)
Y4 = None #Alternatively: series for Y4 (analogous to Y1)
X5 = None #Alternatively: series for X5 (analogous to X1)
Y5 = None #Alternatively: series for Y5 (analogous to Y1)
X6 = None #Alternatively: series for X6 (analogous to X1)
Y6 = None #Alternatively: series for Y6 (analogous to Y1)
# Warning: if X2, X3, X4, X5, and X6 were timestamps, do not forget to use the method
# .astype('datetime64[D]'). e.g.: X2 = (DATASET['DATE']).astype('datetime64[D]')
# If all X axis are the same, you can also declare: X2 = X1, X3 = X1, X4 = X1, X5 = X1
# and X6 = X1.

X_AXIS_ROTATION = 70
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.
ADD_SPLINE_LINES = True #Alternatively: True or False
# If ADD_SPLINE_LINES = False, no lines connecting the successive values are shown.
ADD_SCATTER_DOTS = False #Alternatively: True or False
# If ADD_SCATTER_DOTS = False, the dots (scatter plot) are omitted, so only the lines
# correspondent to the series are shown.

# Notice that adding the dots and omitting the spline lines is equivalent to obtain a
# scatter plot. If you want to do so, consider using the scatter_plot_lin_reg function, 
# capable of calculating the linear regressions.

LAB1 = None #Alternatively: string inside quotes containing the label for series 1
LAB2 = None #Alternatively: string inside quotes containing the label for series 2
LAB3 = None #Alternatively: string inside quotes containing the label for series 3
LAB4 = None #Alternatively: string inside quotes containing the label for series 4
LAB5 = None #Alternatively: string inside quotes containing the label for series 5
LAB6 = None #Alternatively: string inside quotes containing the label for series 6
#e.g. LAB1 = "Y1_values"

HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'time_series_vis.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

time_series_vis (x1 = X1, y1 = Y1, x2 = X2, y2 = Y2, x3 = X3, y3 = Y3, x4 = X4, y4 = Y4, x5 = X5, y5 = Y5, x6 = X6, y6 = Y6, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, add_splines_lines = ADD_SPLINE_LINES, add_scatter_dots = ADD_SCATTER_DOTS, lab1 = LAB1, lab2 = LAB2, lab3 = LAB3, lab4 = LAB4, lab5 = LAB5, lab6 = LAB6, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Visualizing histograms**

#### Case 1: automatically calculate the ideal histogram bin size
- The ideal bin interval is calculated through Montgomery's method. Histogram is obtained from this calculated bin size.
    - Douglas C. Montgomery (2009). Introduction to Statistical Process Control, Sixth Edition, John Wiley & Sons.

In [ ]:
# REMEMBER: A histogram is the representation of a statistical distribution 
# of a given variable.

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

ANALYZED_VARIABLE = DATASET['analyzed_variable']
#Alternatively: other column in quotes, substituting 'analyzed_variable'
# e.g., if the analyzed variable is in a column named 'column1':
# ANALYZED_VARIABLE = DATASET['column1']

SET_GRAPHIC_BAR_WIDTH = 2.0
# This parameter must be visually adjusted for each particular analyzed variable.
# Manually set this parameter until you see only a minimal separation between successive
# bars (i.e., you know that the bars are not overlapping, but they are not so distant that
# the statistic distribution profile is not clear).
# You can input any numeric value, and the order of magnitude will vary depending on the
# dispersion and on the width of the sample space.
# e.g. SET_GRAPHIC_BAR_WIDTH = 3; SET_GRAPHIC_BAR_WIDTH = 0.003

X_AXIS_ROTATION = 70 
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

Y_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

NORMAL_CURVE_OVERLAY = True
#Alternatively: set NORMAL_CURVE_OVERLAY = True to show a normal curve overlaying the
# histogram; or set NORMAL_CURVE_OVERLAY = False to omit the normal curve (show only
# the histogram).

DATA_UNITS_LABEL = None
# Input a string inside quotes for setting a label for the X-axis, that will represent
# The type of data that the histogram is evaluating, i.e., what the statistic distribution
# shown by the histogram means.
# e.g. if DATA_UNITS_LABEL = "particle_diameter_in_nm", the axis X will be labelled with
# this string. Then, we can know that the diagram represents the distribution (counts of
# data for each defined bin) of particles diameters.

Y_TITLE = None
#Alternatively: string inside quotes for vertical title. e.g. Y_TITLE = "Analyzed_values".

HISTOGRAM_TITLE = None
#Alternatively: string inside quotes for graphic title. e.g. 
# HISTOGRAM_TITLE = "Analyzed_values_histogram".

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'histogram.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframes saved as general_statistics and frequency_tab.
# Simply modify this object on the left of equality:
general_statistics, frequency_tab = histogram (y = ANALYZED_VARIABLE, bar_width = SET_GRAPHIC_BAR_WIDTH, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, normal_curve_overlay = NORMAL_CURVE_OVERLAY, data_units_label = DATA_UNITS_LABEL, y_title = Y_TITLE, histogram_title = HISTOGRAM_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

# Suppose we registered the values corresponding to a given feature / property / attribute
# in column Y, and we want to know the Y statistic distribution. the maximum value observed
# for Y is named Ymax, whereas the minimum value observed is Ymin.
# Therefore, our sample space ranges from Ymin to Ymax. Now, we divide this sample space into
# equally-separated intervals, named bins. The width of each bin is the bin_size. The 1st bin
# corresponds to the interval Ymin to (Ymin + bin_size), where we can call (Ymin + bin_size)
# = Y1. Then, the 2nd interval ranges from Y1 to (Y1 + bin_size),..., until we get to the
# last interval, where we find Ymax.
# Now, we count how many values of Y belong to each bin. The graphic of count of values in
# each bin x the bin interval (or the value correspondent to the half of the bin) is the
# histogram. For the first bin this mid-value would be Ymin + bin_size/2, since this value is
# exactly in the middle of interval Ymin to (Ymin + bin_size).

# In other words we can imagine that each Y value was print on the surface of a ball, and
# each bin is a bucket labelled Ymin - (Ymin + bin_size), (Ymin + bin_size) - 
# (Ymin + 2bin_size), untill we cover the Ymax value. We put every ball inside the bucket,
# given that the ball value must be in the interval labelling the bucket. Finally, we count
# the balls per bucket, and plot count of balls x (the middle value of the interval 
# labelling the correspondent bucket). This graphic will be the histogram and will 
# represent the statistical distribution.

#### Case 2: set number of bins
- Use this one if the distance between data is too small, or if the histogram function did not return a valid histogram.
- Here, the histogram is obtained by manually defining the total of bins (i.e., into how much intervals the sample space should be divided).

In [ ]:
# REMEMBER: A histogram is the representation of a statistical distribution 
# of a given variable.

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

ANALYZED_VARIABLE = DATASET['analyzed_variable']
#Alternatively: other column in quotes, substituting 'analyzed_variable'
# e.g., if the analyzed variable is in a column named 'column1':
# ANALYZED_VARIABLE = DATASET['column1']

TOTAL_OF_BINS = 50
# This parameter must be an integer number: it represents the total of bins of the 
# histogram, i.e., the number of divisions of the sample space (in how much intervals
# the sample space will be divided. Check comments after the histogram_alternative
# function call).
# Manually adjust this parameter to obtain more or less resolution of the statistical
# distribution: less bins tend to result into higher counting of values per bin, since
# a larger interval of values is grouped. After modifying the total of bins, do not forget
# to adjust the bar width in SET_GRAPHIC_BAR_WIDTH.
# Examples: TOTAL_OF_BINS = 50, to divide the sample space into 50 equally-separated 
# intervals; TOTAL_OF_BINS = 10 to divide it into 10 intervals; TOTAL_OF_BINS = 100 to
# divide it into 100 intervals.

SET_GRAPHIC_BAR_WIDTH = 2.0
# This parameter must be visually adjusted for each particular analyzed variable.
# Manually set this parameter until you see only a minimal separation between successive
# bars (i.e., you know that the bars are not overlapping, but they are not so distant that
# the statistic distribution profile is not clear).
# You can input any numeric value, and the order of magnitude will vary depending on the
# dispersion and on the width of the sample space.
# e.g. SET_GRAPHIC_BAR_WIDTH = 3; SET_GRAPHIC_BAR_WIDTH = 0.003

X_AXIS_ROTATION = 70 
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

Y_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

DATA_UNITS_LABEL = None
# Input a string inside quotes for setting a label for the X-axis, that will represent
# The type of data that the histogram is evaluating, i.e., what the statistic distribution
# shown by the histogram means.
# e.g. if DATA_UNITS_LABEL = "particle_diameter_in_nm", the axis X will be labelled with
# this string. Then, we can know that the diagram represents the distribution (counts of
# data for each defined bin) of particles diameters.

Y_TITLE = None
#Alternatively: string inside quotes for vertical title. e.g. Y_TITLE = "Analyzed_values".

HISTOGRAM_TITLE = None
#Alternatively: string inside quotes for graphic title. e.g. 
# HISTOGRAM_TITLE = "Analyzed_values_histogram".

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'histogram_alternative.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframes saved as general_statistics and frequency_tab.
# Simply modify this object on the left of equality:
general_statistics, frequency_tab = histogram_alternative (y = ANALYZED_VARIABLE, total_of_bins = TOTAL_OF_BINS, bar_width = SET_GRAPHIC_BAR_WIDTH, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, data_units_label = DATA_UNITS_LABEL, y_title = Y_TITLE, histogram_title = HISTOGRAM_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

# Suppose we registered the values corresponding to a given feature / property / attribute
# in column Y, and we want to know the Y statistic distribution. the maximum value observed
# for Y is named Ymax, whereas the minimum value observed is Ymin.
# Therefore, our sample space ranges from Ymin to Ymax. Now, we divide this sample space into
# equally-separated intervals, named bins. The width of each bin is the bin_size. The 1st bin
# corresponds to the interval Ymin to (Ymin + bin_size), where we can call (Ymin + bin_size)
# = Y1. Then, the 2nd interval ranges from Y1 to (Y1 + bin_size),..., until we get to the
# last interval, where we find Ymax.
# Now, we count how many values of Y belong to each bin. The graphic of count of values in
# each bin x the bin interval (or the value correspondent to the half of the bin) is the
# histogram. For the first bin this mid-value would be Ymin + bin_size/2, since this value is
# exactly in the middle of interval Ymin to (Ymin + bin_size).

# In other words we can imagine that each Y value was print on the surface of a ball, and
# each bin is a bucket labelled Ymin - (Ymin + bin_size), (Ymin + bin_size) - 
# (Ymin + 2bin_size), untill we cover the Ymax value. We put every ball inside the bucket,
# given that the ball value must be in the interval labelling the bucket. Finally, we count
# the balls per bucket, and plot count of balls x (the middle value of the interval 
# labelling the correspondent bucket). This graphic will be the histogram and will 
# represent the statistical distribution.

## **Exporting the dataframe as CSV file**

In [ ]:
## WARNING: all file extensions should be .csv for this function

DATAFRAME_TO_BE_EXPORTED = dataset
#Alternatively: object containing the dataset to be exported.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITH_CSV_EXTENSION = "dataset.csv"
# NEW_FILE_NAME_WITH_CSV_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_CSV_EXTENSION = "file.csv"

EXPORT_TO_S3_BUCKET = False
# export_to_s3_bucket = False. Alternatively, set as True to export the file to an
# AWS S3 Bucket.
    
## The following parameters have effect only when export_to_s3_bucket == True:

S3_BUCKET_NAME = None    
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"
DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION = None
# The name desired for the object stored in S3 (string, in quotes). 
# Keep it None to set it equals to NEW_FILE_NAME_WITH_CSV_EXTENSION. 
# Alternatively, set it as a string analogous to NEW_FILE_NAME_WITH_CSV_EXTENSION.
# e.g. DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION = "S3_file.csv"

export_dataframe(dataframe_to_be_exported = DATAFRAME_TO_BE_EXPORTED, new_file_name_with_csv_extension = NEW_FILE_NAME_WITH_CSV_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH, export_to_s3_bucket = EXPORT_TO_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, desired_s3_file_name_with_csv_extension = DESIRED_S3_FILE_NAME_WITH_CSV_EXTENSION)

## **Downloading a file from Google Colab or AWS S3 to the local machine or uploading a file from the machine to S3 or to Colab's instant memory**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for downloading from (or uploading to) Google Colab's instant memory;
# SOURCE = 'aws' for downloading from (or uploading to) an AWS S3 bucket.

ACTION = 'download'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to AWS S3 or to Google Colab's 
# instant memory

OBJECT_TO_DOWNLOAD_FROM_COLAB = None
# OBJECT_TO_DOWNLOAD_FROM_COLAB = None. This option has effect only when
# SOURCE == 'google'. In this case, this parameter is obbligatory. 
# Declare as OBJECT_TO_DOWNLOAD_FROM_COLAB the object that you want to download.
# Since it is an object and not a string, it should not be declared in quotes.
# e.g. to download a dictionary named dict, OBJECT_TO_DOWNLOAD_FROM_COLAB = dict.
# To download a dataframe named df, declare OBJECT_TO_DOWNLOAD_FROM_COLAB = df.
# To export a model named keras_model, declare OBJECT_TO_DOWNLOAD_FROM_COLAB = keras_model
    
## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'

S3_BUCKET_NAME = None
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. S3_BUCKET_NAME = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

LOCAL_PATH_OF_STORAGE = '/'
# LOCAL_PATH_OF_STORAGE: path of the local computer environment 
# to which the S3 bucket contents will be downloaded (ACTION == 'download'); or
# path of the folder containing the file that will be uploaded in S3 (ACTION = 'upload'). 
# If it is None, or if LOCAL_PATH_OF_STORAGE = '/', files 
# will be imported to the root path. Alternatively, input the path as a string (in quotes). 
# Examples: LOCAL_PATH_OF_STORAGE = '/copied_s3_bucket'; 
# LOCAL_PATH_OF_STORAGE = "/My_folder"; LOCAL_PATH_OF_STORAGE = "/Users/Me/Documents/"
# Notice that only the directories should be declared: do not include the file name and
# its extension.

FILE_NAME_WITH_EXTENSION = None
# FILE_NAME_WITH_EXTENSION: string, in quotes, containing the file name which will be
# downloaded from S3; or uploaded from S3, followed by its extension. 
## This parameter is obbligatory when SOURCE == 'aws'
# Examples:
# FILE_NAME_WITH_EXTENSION = 'Screen_Shot.png'; FILE_NAME_WITH_EXTENSION = 'dataset.csv',
# FILE_NAME_WITH_EXTENSION = "dictionary.pkl", FILE_NAME_WITH_EXTENSION = "model.h5",
# FILE_NAME_WITH_EXTENSION = 'doc.pdf', FILE_NAME_WITH_EXTENSION = 'model.dill'

download_or_upload_file (source = SOURCE, action = ACTION, object_to_download_from_colab = OBJECT_TO_DOWNLOAD_FROM_COLAB, s3_bucket_name = S3_BUCKET_NAME, local_path_of_storage = LOCAL_PATH_OF_STORAGE, file_name_with_extension = FILE_NAME_WITH_EXTENSION)

****